In [ ]:
# import pandas as pd
# import os
# import torch
# from model.database_util import *

In [2]:
# Base directory name
base_dir = 'job_queries'

# Dec 1:
# query1_ts = '2024-10-08-04.50.20.045775' # - worked
#query1_ts = '2024-12-03-20.44.07.528574' # worked
query1_ts = '2024-12-03-20.51.21.351778' # worked
# query1_ts = '2024-12-03-20.53.47.246561'

#query1_ts = '2024-10-08-04.48.13.056756'
# 408
#query1_ts = '2024-10-08-10.06.01.048516' - working but not filtered
# query1_ts = '2024-10-08-04.48.39.202520'
#query1_ts = '2024-10-08-04.48.55.567842'
# query1_ts = '2024-10-08-04.51.06.406285' # not working
# query1_ts = '2024-10-08-10.06.01.048516' # worked in 0 notebook, also worked here


In [3]:


# File paths for the query explanation components
file_paths = {
    'operator': f'{base_dir}/EXPLAIN_OPERATOR.csv',
    'stream': f'{base_dir}/EXPLAIN_STREAM.csv',
    'predicate': f'{base_dir}/EXPLAIN_PREDICATE.csv'
}


In [4]:
max_node = 30
rel_pos_max = 20

# Extract the operator ids and types

In [5]:
import pandas as pd
import os

def extract_operator_ids_and_types(explain_operator_path, timestamp):
    """
    Extract rows from a CSV file where the EXPLAIN_TIME column matches a given timestamp.

    Parameters:
        explain_operator_path (str): Path to the CSV file.
        timestamp: The timestamp value to filter rows by.

    Returns:
        pd.DataFrame: A DataFrame containing the matching rows with EXPLAIN_TIME, OPERATOR_ID, and OPERATOR_TYPE.
    """
    # Check if the file exists
    if not os.path.exists(explain_operator_path):
        raise FileNotFoundError(f"The file '{explain_operator_path}' does not exist.")
    
    # Read the CSV file
    try:
        df = pd.read_csv(explain_operator_path)
    except Exception as e:
        raise ValueError(f"Error reading the file: {e}")

    # Check if the required columns exist
    required_columns = ['EXPLAIN_TIME', 'OPERATOR_ID', 'OPERATOR_TYPE']
    if not all(col in df.columns for col in required_columns):
        raise KeyError(f"The file must contain the columns: {required_columns}")

    # Filter rows where the EXPLAIN_TIME column matches the timestamp value
    df_ops = df[df['EXPLAIN_TIME'] == timestamp][required_columns]

    # Warn if no rows match the timestamp
    if df_ops.empty:
        print(f"No rows found for timestamp '{timestamp}'.")

    return df_ops


# try:
#     result = extract_operator_ids_and_types(explain_operator_path, query1_ts)
#     print(result)
# except Exception as e:
#     print(f"Error: {e}")


# Extract the Operators accessing objects

In [6]:
import pandas as pd

def extract_stream_table(explain_stream_path, timestamp):
    """
    Process the EXPLAIN_STREAM CSV file to extract relevant rows, concatenate schema and name, and prepare the output.

    Parameters:
        explain_stream_path (str): Path to the EXPLAIN_STREAM CSV file.
        timestamp (str): The timestamp value to filter rows by.

    Returns:
        pd.DataFrame: A DataFrame containing EXPLAIN_TIME, OPERATOR_ID, and TABLE columns.
    """
    # Read the CSV file
    df = pd.read_csv(explain_stream_path)

    # Filter for the given EXPLAIN_TIME value and non-NaN OBJECT_NAME
    filtered_df = df.loc[(df['EXPLAIN_TIME'] == timestamp) & (df['OBJECT_NAME'].notna())].copy()

    # Concatenate OBJECT_SCHEMA and OBJECT_NAME to create a new column, removing any extra spaces
    filtered_df['TABLE'] = (
        filtered_df['OBJECT_SCHEMA'].str.strip().fillna('') +
        '.' +
        filtered_df['OBJECT_NAME'].str.strip()
    )

    # Select and rename the TARGET_ID column to OPERATOR_ID
    df_table = filtered_df[['EXPLAIN_TIME', 'SOURCE_ID', 'TARGET_ID', 'TABLE']].copy()
    df_table.rename(columns={'TARGET_ID': 'OPERATOR_ID'}, inplace=True)

    return df_table


# Example usage:
explain_stream_path = f'{base_dir}/EXPLAIN_STREAM.csv'

query1_ts = '2024-12-03-20.44.07.528574'  # Replace with your timestamp
result = extract_stream_table(explain_stream_path, query1_ts)
print(result)


                    EXPLAIN_TIME  SOURCE_ID  OPERATOR_ID  \
6388  2024-12-03-20.44.07.528574         -1            4   
6390  2024-12-03-20.44.07.528574         -1            6   
6392  2024-12-03-20.44.07.528574         -1            8   
6394  2024-12-03-20.44.07.528574         -1           11   
6396  2024-12-03-20.44.07.528574         -1           14   
6398  2024-12-03-20.44.07.528574         -1           16   
6400  2024-12-03-20.44.07.528574         -1           17   
6404  2024-12-03-20.44.07.528574         -1           18   
6408  2024-12-03-20.44.07.528574         -1           19   

                      TABLE  
6388            AHNAF.TITLE  
6390       AHNAF.MOVIE_INFO  
6392     AHNAF.COMPANY_NAME  
6394  AHNAF.MOVIE_COMPANIES  
6396       AHNAF.MOVIE_LINK  
6398    AHNAF.MOVIE_KEYWORD  
6400          AHNAF.KEYWORD  
6404        AHNAF.LINK_TYPE  
6408     AHNAF.COMPANY_TYPE  


# Extract local predicates

In [7]:
import pandas as pd

def extract_local_predicates(explain_predicate_path, timestamp):
    """
    Extract and process local predicates (WHERE/HAVING conditions) from the EXPLAIN_PREDICATE CSV file.

    Parameters:
        explain_predicate_path (str): Path to the EXPLAIN_PREDICATE CSV file.
        timestamp (str): The timestamp value to filter rows by.

    Returns:
        pd.DataFrame: A DataFrame with grouped local predicates (PREDICATE1, PREDICATE2, PREDICATE3) 
                      and their corresponding FILTER_FACTOR columns (FILTER_FACTOR1, FILTER_FACTOR2, FILTER_FACTOR3) 
                      for each EXPLAIN_TIME and OPERATOR_ID.
    """
    # Define the columns to read from the file
    columns_to_read = ['EXPLAIN_TIME', 'OPERATOR_ID', 'HOW_APPLIED', 'PREDICATE_TEXT', 'FILTER_FACTOR']

    # Read the CSV file, selecting only relevant columns
    df = pd.read_csv(explain_predicate_path, usecols=columns_to_read)

    # Strip whitespace from EXPLAIN_TIME and HOW_APPLIED columns
    df['EXPLAIN_TIME'] = df['EXPLAIN_TIME'].str.strip()
    df['HOW_APPLIED'] = df['HOW_APPLIED'].str.strip()

    # Filter rows where EXPLAIN_TIME matches the timestamp and HOW_APPLIED is 'SARG'
    filtered_df = df.loc[(df['EXPLAIN_TIME'] == timestamp) & (df['HOW_APPLIED'] == 'SARG')].copy()

    # Remove only outer parentheses from the PREDICATE_TEXT column
    def remove_outer_parentheses(predicate_text):
        # Check if the text starts and ends with parentheses
        if isinstance(predicate_text, str) and predicate_text.startswith('(') and predicate_text.endswith(')'):
            # Remove only the outermost parentheses
            return predicate_text[1:-1]
        return predicate_text

    filtered_df['PREDICATE_TEXT'] = filtered_df['PREDICATE_TEXT'].apply(remove_outer_parentheses)

    # Select the relevant columns for the result
    df_predicates = filtered_df[['EXPLAIN_TIME', 'OPERATOR_ID', 'PREDICATE_TEXT', 'FILTER_FACTOR']]

    # Group by EXPLAIN_TIME and OPERATOR_ID and apply transformation
    df_local_predicates = df_predicates.groupby(['EXPLAIN_TIME', 'OPERATOR_ID']).agg(
        PREDICATE1=('PREDICATE_TEXT', lambda x: x.iloc[0] if len(x) > 0 else None),
        PREDICATE2=('PREDICATE_TEXT', lambda x: x.iloc[1] if len(x) > 1 else None),
        PREDICATE3=('PREDICATE_TEXT', lambda x: x.iloc[2] if len(x) > 2 else None),
        FILTER_FACTOR1=('FILTER_FACTOR', lambda x: x.iloc[0] if len(x) > 0 else None),
        FILTER_FACTOR2=('FILTER_FACTOR', lambda x: x.iloc[1] if len(x) > 1 else None),
        FILTER_FACTOR3=('FILTER_FACTOR', lambda x: x.iloc[2] if len(x) > 2 else None),
    ).reset_index()
    
    # Replace NaN values in FILTER_FACTOR columns with 1.0
    filter_factor_columns = ['FILTER_FACTOR1', 'FILTER_FACTOR2', 'FILTER_FACTOR3']
    df_local_predicates[filter_factor_columns] = df_local_predicates[filter_factor_columns].fillna(1.0)

    return df_local_predicates


# Example usage:
# explain_predicate_path = 'SimpleQueriesSQ12c/EXPLAIN_PREDICATE.csv'
# query1_ts = '2024-11-28 12:00:00'  # Replace with your timestamp
explain_predicate_path = f'{base_dir}/EXPLAIN_PREDICATE.csv'
result = extract_local_predicates(explain_predicate_path, query1_ts)
print(result)


                 EXPLAIN_TIME  OPERATOR_ID  \
0  2024-12-03-20.44.07.528574            4   
1  2024-12-03-20.44.07.528574            6   
2  2024-12-03-20.44.07.528574            8   
3  2024-12-03-20.44.07.528574           11   
4  2024-12-03-20.44.07.528574           17   
5  2024-12-03-20.44.07.528574           18   
6  2024-12-03-20.44.07.528574           19   

                                          PREDICATE1  \
0                         Q1.PRODUCTION_YEAR <= 2020   
1  Q4.INFO IN ('Sweden               ', 'Norway  ...   
2                           Q9.COUNTRY_CODE = '[us]'   
3                                   Q5.NOTE IS NULL    
4                          Q7.KEYWORD = 'helicopter'   
5                       Q6.LINK LIKE '%featured in%'   
6                           Q8.KIND = 'distributors'   

                   PREDICATE2 PREDICATE3  FILTER_FACTOR1  FILTER_FACTOR2  \
0  1950 <= Q1.PRODUCTION_YEAR       None        0.971485        0.894801   
1                        None 

/tmp/ipykernel_854004/3940385309.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_local_predicates[filter_factor_columns] = df_local_predicates[filter_factor_columns].fillna(1.0)


In [8]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   EXPLAIN_TIME    7 non-null      object 
 1   OPERATOR_ID     7 non-null      int64  
 2   PREDICATE1      7 non-null      object 
 3   PREDICATE2      1 non-null      object 
 4   PREDICATE3      0 non-null      object 
 5   FILTER_FACTOR1  7 non-null      float64
 6   FILTER_FACTOR2  7 non-null      float64
 7   FILTER_FACTOR3  7 non-null      float64
dtypes: float64(3), int64(1), object(4)
memory usage: 580.0+ bytes


In [9]:
result.head(3)

,EXPLAIN_TIME,OPERATOR_ID,PREDICATE1,PREDICATE2,PREDICATE3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3
0,2024-12-03-20.44.07.528574,4,Q1.PRODUCTION_YEAR <= 2020,1950 <= Q1.PRODUCTION_YEAR,None,0.971485,0.894801,1.0
1,2024-12-03-20.44.07.528574,6,"Q4.INFO IN ('Sweden ', 'Norway ...",None,None,0.029517,1.000000,1.0
2,2024-12-03-20.44.07.528574,8,Q9.COUNTRY_CODE = '[us]',None,None,0.359728,1.000000,1.0


In [10]:
result[['OPERATOR_ID', 'PREDICATE1', 'PREDICATE2', 
       'FILTER_FACTOR1', 'FILTER_FACTOR2']]

,OPERATOR_ID,PREDICATE1,PREDICATE2,FILTER_FACTOR1,FILTER_FACTOR2
0,4,Q1.PRODUCTION_YEAR <= 2020,1950 <= Q1.PRODUCTION_YEAR,0.971485,0.894801
1,6,"Q4.INFO IN ('Sweden ', 'Norway ...",None,0.029517,1.000000
2,8,Q9.COUNTRY_CODE = '[us]',None,0.359728,1.000000
3,11,Q5.NOTE IS NULL,None,0.487515,1.000000
4,17,Q7.KEYWORD = 'helicopter',None,0.000007,1.000000
5,18,Q6.LINK LIKE '%featured in%',None,0.072222,1.000000
6,19,Q8.KIND = 'distributors',None,0.250000,1.000000


In [11]:
pd.set_option('display.max_colwidth', None)

# Parsing the PREDICATEs and adding PARSED_PREDICATE columns

In [12]:
import re

def parse_predicate(predicate):
    """
    Parse a predicate string into a tuple (col, op, val).
    Supports various predicate types including IN, LIKE, IS NULL, and comparisons.
    """
    print('Original predicate:', predicate)
    if predicate is None or pd.isna(predicate):
        return None

    predicate = predicate.strip()

    # Patterns for different predicate types
    patterns = {
        'basic': r'(?P<col>[\w\.]+)\s*(?P<op>=|<=|>=|<|>)\s*(?P<val>.+)',
        'in': r'(?P<col>[\w\.]+)\s+IN\s+\((?P<val>.+?)\)',
        'like': r'(?P<col>[\w\.]+)\s+LIKE\s+(?P<val>.+)',
        'is': r'(?P<col>[\w\.]+)\s+IS\s+(?P<val>NULL|NOT\s+NULL)'
    }

    # Match and parse based on patterns
    for key, pattern in patterns.items():
        match = re.match(pattern, predicate, flags=re.IGNORECASE)
        if match:
            print(f"Matched pattern: {key} | Groups: {match.groupdict()}")
            col = match.group('col').strip()
            
            col_parts = col.split('.')
            if len(col_parts) == 3:
                col = '.'.join(col_parts[1:])
            
            op = match.group('op')
            val = match.group('val').strip()

            # Handle special cases
            if key == 'in':
                # Split and clean IN clause values
                val = [
                    f"'{v.strip()}'" for v in val.split(',')
                ]
            elif key == 'basic' and val.startswith("'") and val.endswith("'"):
                # Clean single-quoted values for basic predicates
                val = val.strip("'")

            return (col, op, val)

    # Log failed predicates
    print(f"Failed to match predicate: {predicate}")
    return None


In [13]:
explain_predicate_path = f'{base_dir}/EXPLAIN_PREDICATE.csv'
# query1_ts = '2024-10-08-04.51.06.406285' # not working

# Use the functions to extract features
df_pred = extract_local_predicates(explain_predicate_path, query1_ts)

/tmp/ipykernel_854004/3940385309.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_local_predicates[filter_factor_columns] = df_local_predicates[filter_factor_columns].fillna(1.0)


In [14]:
df_pred

,EXPLAIN_TIME,OPERATOR_ID,PREDICATE1,PREDICATE2,PREDICATE3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3
0,2024-12-03-20.44.07.528574,4,Q1.PRODUCTION_YEAR <= 2020,1950 <= Q1.PRODUCTION_YEAR,None,0.971485,0.894801,1.0
1,2024-12-03-20.44.07.528574,6,"Q4.INFO IN ('Sweden ', 'Norway ', 'Germany ', 'Denmark ', 'Swedish ', 'Denish ', 'Norwegian ', 'German ', 'USA ', 'CANADA ', 'Netherlands ', 'Brazil ', 'UK ', 'Belgium ', 'Finland ', 'Hungary ', 'Estonia ', 'worldwide ', 'Australia ', 'Spain ', 'France ', 'Japan ', 'Columbia ', 'Slovenia ', 'Israel ', 'Venezuela ', 'Nigeria ', 'Philippines ', 'New Zealand ', 'Ireland ', 'Romania ', 'Non-USA ', 'Bulgaria ', 'Argentina ', 'Malaysia ', 'Singapore ', 'Turkey ', 'Sri Lanka ', 'Italy ', 'Indonesia ', 'South Korea ', 'Vietnam ', 'Slovakia ', 'Czech Republic ', 'China ', 'Portugal ', 'Greece ', 'Republic of Macedonia', 'Serbia ', 'Jamaica ', 'Switzerland ', 'Yugoslavia ', 'Mexico ', 'Austria ', 'Russia ')",None,None,0.029517,1.000000,1.0
2,2024-12-03-20.44.07.528574,8,Q9.COUNTRY_CODE = '[us]',None,None,0.359728,1.000000,1.0
3,2024-12-03-20.44.07.528574,11,Q5.NOTE IS NULL,None,None,0.487515,1.000000,1.0
4,2024-12-03-20.44.07.528574,17,Q7.KEYWORD = 'helicopter',None,None,0.000007,1.000000,1.0
5,2024-12-03-20.44.07.528574,18,Q6.LINK LIKE '%featured in%',None,None,0.072222,1.000000,1.0
6,2024-12-03-20.44.07.528574,19,Q8.KIND = 'distributors',None,None,0.250000,1.000000,1.0


In [15]:
import re

def parse_predicate(predicate):
    print('Original predicate:', predicate)
    if predicate is None or pd.isna(predicate):
        return None

    # Match basic predicate (col, op, val)
    basic_pattern = r'(?P<col>[\w\.]+)\s*(?P<op><=|>=|<|>|=|!=|LIKE)\s*(?P<val>.+)'
    # Match IN predicate
    in_pattern = r'(?P<col>[\w\.]+)\s+IN\s+\((?P<val>.+?)\)'

    # Try to match basic predicate
    match = re.match(basic_pattern, predicate.strip())
    if match:
        col = match.group('col')
        op = match.group('op')
        val = match.group('val')
        return (col, op, val)

    # Try to match IN predicate
    match = re.match(in_pattern, predicate.strip(), flags=re.IGNORECASE)
    if match:
        col = match.group('col')
        op = 'IN'
        val = match.group('val')

        # Debugging: Log the raw values before processing
        print('Raw IN values:', val)

        # Split by commas and clean spaces inside quotes
        val_list = [
            f'{v.strip().strip('\'').strip()}' if v.strip().startswith("'") else v.strip()
            for v in val.split(',')
        ]

        # Debugging: Log the cleaned values after processing
        print('Cleaned IN values:', val_list)

        return (col, op, val_list)

    return None

predicate = "AHNAF.MOVIE_INFO.INFO IN (' Sweden ', 'Norway ', ' Germany', ' Denmark ')"
result = parse_predicate(predicate)
print('Parsed result:', result)


Original predicate: AHNAF.MOVIE_INFO.INFO IN (' Sweden ', 'Norway ', ' Germany', ' Denmark ')
Raw IN values: ' Sweden ', 'Norway ', ' Germany', ' Denmark '
Cleaned IN values: ['Sweden', 'Norway', 'Germany', 'Denmark']
Parsed result: ('AHNAF.MOVIE_INFO.INFO', 'IN', ['Sweden', 'Norway', 'Germany', 'Denmark'])


In [16]:
df_pred

,EXPLAIN_TIME,OPERATOR_ID,PREDICATE1,PREDICATE2,PREDICATE3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3
0,2024-12-03-20.44.07.528574,4,Q1.PRODUCTION_YEAR <= 2020,1950 <= Q1.PRODUCTION_YEAR,None,0.971485,0.894801,1.0
1,2024-12-03-20.44.07.528574,6,"Q4.INFO IN ('Sweden ', 'Norway ', 'Germany ', 'Denmark ', 'Swedish ', 'Denish ', 'Norwegian ', 'German ', 'USA ', 'CANADA ', 'Netherlands ', 'Brazil ', 'UK ', 'Belgium ', 'Finland ', 'Hungary ', 'Estonia ', 'worldwide ', 'Australia ', 'Spain ', 'France ', 'Japan ', 'Columbia ', 'Slovenia ', 'Israel ', 'Venezuela ', 'Nigeria ', 'Philippines ', 'New Zealand ', 'Ireland ', 'Romania ', 'Non-USA ', 'Bulgaria ', 'Argentina ', 'Malaysia ', 'Singapore ', 'Turkey ', 'Sri Lanka ', 'Italy ', 'Indonesia ', 'South Korea ', 'Vietnam ', 'Slovakia ', 'Czech Republic ', 'China ', 'Portugal ', 'Greece ', 'Republic of Macedonia', 'Serbia ', 'Jamaica ', 'Switzerland ', 'Yugoslavia ', 'Mexico ', 'Austria ', 'Russia ')",None,None,0.029517,1.000000,1.0
2,2024-12-03-20.44.07.528574,8,Q9.COUNTRY_CODE = '[us]',None,None,0.359728,1.000000,1.0
3,2024-12-03-20.44.07.528574,11,Q5.NOTE IS NULL,None,None,0.487515,1.000000,1.0
4,2024-12-03-20.44.07.528574,17,Q7.KEYWORD = 'helicopter',None,None,0.000007,1.000000,1.0
5,2024-12-03-20.44.07.528574,18,Q6.LINK LIKE '%featured in%',None,None,0.072222,1.000000,1.0
6,2024-12-03-20.44.07.528574,19,Q8.KIND = 'distributors',None,None,0.250000,1.000000,1.0


In [17]:
df_pred

,EXPLAIN_TIME,OPERATOR_ID,PREDICATE1,PREDICATE2,PREDICATE3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3
0,2024-12-03-20.44.07.528574,4,Q1.PRODUCTION_YEAR <= 2020,1950 <= Q1.PRODUCTION_YEAR,None,0.971485,0.894801,1.0
1,2024-12-03-20.44.07.528574,6,"Q4.INFO IN ('Sweden ', 'Norway ', 'Germany ', 'Denmark ', 'Swedish ', 'Denish ', 'Norwegian ', 'German ', 'USA ', 'CANADA ', 'Netherlands ', 'Brazil ', 'UK ', 'Belgium ', 'Finland ', 'Hungary ', 'Estonia ', 'worldwide ', 'Australia ', 'Spain ', 'France ', 'Japan ', 'Columbia ', 'Slovenia ', 'Israel ', 'Venezuela ', 'Nigeria ', 'Philippines ', 'New Zealand ', 'Ireland ', 'Romania ', 'Non-USA ', 'Bulgaria ', 'Argentina ', 'Malaysia ', 'Singapore ', 'Turkey ', 'Sri Lanka ', 'Italy ', 'Indonesia ', 'South Korea ', 'Vietnam ', 'Slovakia ', 'Czech Republic ', 'China ', 'Portugal ', 'Greece ', 'Republic of Macedonia', 'Serbia ', 'Jamaica ', 'Switzerland ', 'Yugoslavia ', 'Mexico ', 'Austria ', 'Russia ')",None,None,0.029517,1.000000,1.0
2,2024-12-03-20.44.07.528574,8,Q9.COUNTRY_CODE = '[us]',None,None,0.359728,1.000000,1.0
3,2024-12-03-20.44.07.528574,11,Q5.NOTE IS NULL,None,None,0.487515,1.000000,1.0
4,2024-12-03-20.44.07.528574,17,Q7.KEYWORD = 'helicopter',None,None,0.000007,1.000000,1.0
5,2024-12-03-20.44.07.528574,18,Q6.LINK LIKE '%featured in%',None,None,0.072222,1.000000,1.0
6,2024-12-03-20.44.07.528574,19,Q8.KIND = 'distributors',None,None,0.250000,1.000000,1.0


In [18]:
import re
import pandas as pd

def parse_predicate(predicate):
    """
    Parses a SQL predicate string to extract the column, operator, and value.

    Args:
        predicate (str): The predicate string to parse.

    Returns:
        tuple: A tuple containing the column, operator, and value, or None if parsing fails.
    """
    if predicate is None or pd.isna(predicate):
        return None

    # Patterns for different predicate types
    basic_pattern = r'(?P<col>[\w\.]+)\s*(?P<op><=|>=|<|>|=|!=|LIKE)\s*(?P<val>.+)'
    in_pattern = r'(?P<col>[\w\.]+)\s+IN\s*\((?P<val>.*)\)'
    is_null_pattern = r'(?P<col>[\w\.]+)\s+IS\s+(?P<val>NULL|NOT\s+NULL)'

    # Remove parentheses around the predicate if present
    predicate = predicate.strip()

    # Match IS NULL / IS NOT NULL predicates
    match = re.match(is_null_pattern, predicate, flags=re.IGNORECASE)
    if match:
        col = match.group('col')
        op = 'IS'
        val = match.group('val')
        return (col, op, val)

    # Match IN predicates (e.g., Q4.INFO IN ('Sweden', 'Norway'))
    match = re.match(in_pattern, predicate, flags=re.IGNORECASE)
    if match:
        col = match.group('col')
        op = 'IN'
        val = match.group('val')

        # Extract single-quoted strings and strip spaces inside each value
        val_list = [v.strip() for v in re.findall(r"'(.*?)'", val)]
        if val_list:
            return (col, op, val_list)

        # Fall back to splitting by commas and stripping spaces
        val_list = [v.strip() for v in val.split(',') if v.strip()]
        return (col, op, val_list)

    # Match basic predicates (e.g., Q1.PRODUCTION_YEAR <= 2020)
    match = re.match(basic_pattern, predicate)
    if match:
        col = match.group('col')
        op = match.group('op')
        val = match.group('val')
        return (col, op, val)

    # Return None if no pattern matches
    return None


In [19]:
# Function to transform predicate to (COLUMN, OPERATOR, VALUE) format
def transform_predicate(predicate):
    if isinstance(predicate, str):
        # Regex to match predicates in the form (VALUE OPERATOR COLUMN)
        match = re.match(r'(\d+|\'[^\']*\'|\"[^\"]*\")\s*(<=|>=|<|>)\s*([A-Za-z_][A-Za-z0-9_\.]*)', predicate)
        if match:
            value, operator, column = match.groups()
            # Reverse the operation and switch operator
            reversed_operator = {'<=': '>=', '>=': '<=', '<': '>', '>': '<'}[operator]
            return f'{column} {reversed_operator} {value}'
    return predicate  # Return unchanged if not a string or no match

# Function to apply transformation only if at least one of the predicates is not NaN
def apply_transformation(row):
    if pd.notna(row['PREDICATE1']) or pd.notna(row['PREDICATE2']) or pd.notna(row['PREDICATE3']):
        row['PREDICATE1'] = transform_predicate(row['PREDICATE1'])
        row['PREDICATE2'] = transform_predicate(row['PREDICATE2'])
        row['PREDICATE3'] = transform_predicate(row['PREDICATE3'])
    return row

# Extract Join Key

In [20]:
import pandas as pd

def extract_join_keys(explain_predicate_path, timestamp):
    """
    Extract join keys from the EXPLAIN_PREDICATE CSV file where HOW_APPLIED is 'JOIN'.

    Parameters:
        explain_predicate_path (str): Path to the EXPLAIN_PREDICATE CSV file.
        timestamp (str): The timestamp value to filter rows by.

    Returns:
        pd.DataFrame: A DataFrame containing EXPLAIN_TIME, OPERATOR_ID, and JOIN_KEY columns.
    """
    # Define the columns to read from the file
    columns_to_read = ['EXPLAIN_TIME', 'OPERATOR_ID', 'HOW_APPLIED', 'PREDICATE_TEXT']

    # Read the CSV file, selecting only relevant columns
    df = pd.read_csv(explain_predicate_path, usecols=columns_to_read)

    # Strip whitespace from EXPLAIN_TIME and HOW_APPLIED columns
    df['EXPLAIN_TIME'] = df['EXPLAIN_TIME'].str.strip()
    df['HOW_APPLIED'] = df['HOW_APPLIED'].str.strip()

    # Filter rows where EXPLAIN_TIME matches the timestamp and HOW_APPLIED is 'JOIN'
    filtered_df = df[(df['EXPLAIN_TIME'] == timestamp) & (df['HOW_APPLIED'] == 'JOIN')].copy()

    # Remove parentheses from the PREDICATE_TEXT column and assign to JOIN_KEY
    filtered_df.loc[:, 'JOIN_KEY'] = filtered_df['PREDICATE_TEXT'].str.replace(r'[\(\)]', '', regex=True)

    # Select the relevant columns for the result
    df_join = filtered_df[['EXPLAIN_TIME', 'OPERATOR_ID', 'JOIN_KEY']]

    # Check if there are matching rows
    if df_join.empty:
        print("No matching rows found.")
        return pd.DataFrame()  # Return an empty DataFrame

    return df_join

# Example usage:
# explain_predicate_path = 'SimpleQueriesSQ12c/EXPLAIN_PREDICATE.csv'
# query1_ts = '2024-11-28 12:00:00'  # Replace with your timestamp
# result = extract_join_keys(explain_predicate_path, query1_ts)
# print(result)


# Combining Node features

In [21]:
import pandas as pd

# Define the file paths and timestamp value
# File paths
explain_operator_path = f'{base_dir}/EXPLAIN_OPERATOR.csv'
explain_stream_path = f'{base_dir}/EXPLAIN_STREAM.csv'
explain_predicate_path = f'{base_dir}/EXPLAIN_PREDICATE.csv'
# query1_ts = '2024-10-08-04.51.06.406285' # not working

# Use the functions to extract features
df_ops = extract_operator_ids_and_types(explain_operator_path, query1_ts)
df_join = extract_join_keys(explain_predicate_path, query1_ts)
df_table = extract_stream_table(explain_stream_path, query1_ts)
df_pred = extract_local_predicates(explain_predicate_path, query1_ts)

# Apply the transformation row by row
df_pred = df_pred.apply(apply_transformation, axis=1)


/tmp/ipykernel_854004/3940385309.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_local_predicates[filter_factor_columns] = df_local_predicates[filter_factor_columns].fillna(1.0)


In [22]:
df_pred

,EXPLAIN_TIME,OPERATOR_ID,PREDICATE1,PREDICATE2,PREDICATE3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3
0,2024-12-03-20.44.07.528574,4,Q1.PRODUCTION_YEAR <= 2020,Q1.PRODUCTION_YEAR >= 1950,None,0.971485,0.894801,1.0
1,2024-12-03-20.44.07.528574,6,"Q4.INFO IN ('Sweden ', 'Norway ', 'Germany ', 'Denmark ', 'Swedish ', 'Denish ', 'Norwegian ', 'German ', 'USA ', 'CANADA ', 'Netherlands ', 'Brazil ', 'UK ', 'Belgium ', 'Finland ', 'Hungary ', 'Estonia ', 'worldwide ', 'Australia ', 'Spain ', 'France ', 'Japan ', 'Columbia ', 'Slovenia ', 'Israel ', 'Venezuela ', 'Nigeria ', 'Philippines ', 'New Zealand ', 'Ireland ', 'Romania ', 'Non-USA ', 'Bulgaria ', 'Argentina ', 'Malaysia ', 'Singapore ', 'Turkey ', 'Sri Lanka ', 'Italy ', 'Indonesia ', 'South Korea ', 'Vietnam ', 'Slovakia ', 'Czech Republic ', 'China ', 'Portugal ', 'Greece ', 'Republic of Macedonia', 'Serbia ', 'Jamaica ', 'Switzerland ', 'Yugoslavia ', 'Mexico ', 'Austria ', 'Russia ')",None,None,0.029517,1.000000,1.0
2,2024-12-03-20.44.07.528574,8,Q9.COUNTRY_CODE = '[us]',None,None,0.359728,1.000000,1.0
3,2024-12-03-20.44.07.528574,11,Q5.NOTE IS NULL,None,None,0.487515,1.000000,1.0
4,2024-12-03-20.44.07.528574,17,Q7.KEYWORD = 'helicopter',None,None,0.000007,1.000000,1.0
5,2024-12-03-20.44.07.528574,18,Q6.LINK LIKE '%featured in%',None,None,0.072222,1.000000,1.0
6,2024-12-03-20.44.07.528574,19,Q8.KIND = 'distributors',None,None,0.250000,1.000000,1.0


In [23]:
df_pred.columns

Index(['EXPLAIN_TIME', 'OPERATOR_ID', 'PREDICATE1', 'PREDICATE2', 'PREDICATE3',
       'FILTER_FACTOR1', 'FILTER_FACTOR2', 'FILTER_FACTOR3'],
      dtype='object')

In [24]:
df_pred.head()

,EXPLAIN_TIME,OPERATOR_ID,PREDICATE1,PREDICATE2,PREDICATE3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3
0,2024-12-03-20.44.07.528574,4,Q1.PRODUCTION_YEAR <= 2020,Q1.PRODUCTION_YEAR >= 1950,None,0.971485,0.894801,1.0
1,2024-12-03-20.44.07.528574,6,"Q4.INFO IN ('Sweden ', 'Norway ', 'Germany ', 'Denmark ', 'Swedish ', 'Denish ', 'Norwegian ', 'German ', 'USA ', 'CANADA ', 'Netherlands ', 'Brazil ', 'UK ', 'Belgium ', 'Finland ', 'Hungary ', 'Estonia ', 'worldwide ', 'Australia ', 'Spain ', 'France ', 'Japan ', 'Columbia ', 'Slovenia ', 'Israel ', 'Venezuela ', 'Nigeria ', 'Philippines ', 'New Zealand ', 'Ireland ', 'Romania ', 'Non-USA ', 'Bulgaria ', 'Argentina ', 'Malaysia ', 'Singapore ', 'Turkey ', 'Sri Lanka ', 'Italy ', 'Indonesia ', 'South Korea ', 'Vietnam ', 'Slovakia ', 'Czech Republic ', 'China ', 'Portugal ', 'Greece ', 'Republic of Macedonia', 'Serbia ', 'Jamaica ', 'Switzerland ', 'Yugoslavia ', 'Mexico ', 'Austria ', 'Russia ')",None,None,0.029517,1.000000,1.0
2,2024-12-03-20.44.07.528574,8,Q9.COUNTRY_CODE = '[us]',None,None,0.359728,1.000000,1.0
3,2024-12-03-20.44.07.528574,11,Q5.NOTE IS NULL,None,None,0.487515,1.000000,1.0
4,2024-12-03-20.44.07.528574,17,Q7.KEYWORD = 'helicopter',None,None,0.000007,1.000000,1.0


In [25]:
df_ops.columns

Index(['EXPLAIN_TIME', 'OPERATOR_ID', 'OPERATOR_TYPE'], dtype='object')

In [26]:
df_ops.shape

(19, 3)

In [27]:
df_table.columns

Index(['EXPLAIN_TIME', 'SOURCE_ID', 'OPERATOR_ID', 'TABLE'], dtype='object')

In [28]:
df_join.columns

Index(['EXPLAIN_TIME', 'OPERATOR_ID', 'JOIN_KEY'], dtype='object')

In [29]:
# Merge DataFrames step-by-step
merged_df = pd.merge(df_ops, df_pred, on=['EXPLAIN_TIME', 'OPERATOR_ID'], how='outer')  # Include all rows
merged_df = pd.merge(merged_df, df_join, on=['EXPLAIN_TIME', 'OPERATOR_ID'], how='outer')
merged_df = pd.merge(merged_df, df_table, on=['EXPLAIN_TIME', 'OPERATOR_ID'], how='outer')
# Ensure all FILTER_FACTOR columns are filled
filter_columns = ['FILTER_FACTOR1', 'FILTER_FACTOR2', 'FILTER_FACTOR3']
merged_df[filter_columns] = merged_df[filter_columns].fillna(1.0)

df = merged_df.copy()


In [30]:
import pandas as pd
import re

# Sample DataFrame provided
# df = pd.DataFrame({
#     'EXPLAIN_TIME': ['2024-10-08-04.48.13.056756', '2024-10-08-04.48.13.056756', '2024-10-08-04.48.13.056756', '2024-10-08-04.48.13.056756'],
#     'OPERATOR_ID': [1, 2, 3, 4],
#     'OPERATOR_TYPE': ['RETURN', 'HSJOIN', 'TBSCAN', 'TBSCAN'],
#     'TABLE': [None, None, 'TPCDS.CUSTOMER', 'TPCDS.DATE_DIM2'],
#     'PREDICATE1': [None, None, None, 'Q2.D_YEAR >= 1958'],
#     'PREDICATE2': [None, None, None, 'Q2.D_MOY = 12'],
#     'PREDICATE3': [None, None, None, None],
#     'JOIN_KEY': [None, 'Q2.D_DATE_SK = Q1.C_FIRST_SHIPTO_DATE_SK', None, None]
# })

# Function to extract the alias from the predicate (before the period)
def extract_alias(predicate):
    if isinstance(predicate, str):
        match = re.search(r'([A-Za-z0-9_]+)\.', predicate)
        if match:
            return match.group(1)  # Return the part before the period (alias)
    return None

# Filter rows where TABLE is not None and at least one PREDICATE column has a value
filtered_df = df[(df['TABLE'].notna()) & (df[['PREDICATE1', 'PREDICATE2', 'PREDICATE3']].notna().any(axis=1))]

# Create a new DataFrame with EXPLAIN_TIME, TABLE, and ALIAS
rows = []
for _, row in filtered_df.iterrows():
    predicates = [row['PREDICATE1'], row['PREDICATE2'], row['PREDICATE3']]
    alias = None
    for predicate in predicates:
        alias = extract_alias(predicate)
        if alias:
            break  # Once an alias is found, no need to check other predicates
    rows.append({'EXPLAIN_TIME': row['EXPLAIN_TIME'], 'TABLE': row['TABLE'], 'ALIAS': alias})

# Create the new dataframe
df_table_alias = pd.DataFrame(rows)

# Display the result
print(df_table_alias)


                 EXPLAIN_TIME                  TABLE ALIAS
0  2024-12-03-20.44.07.528574            AHNAF.TITLE    Q1
1  2024-12-03-20.44.07.528574       AHNAF.MOVIE_INFO    Q4
2  2024-12-03-20.44.07.528574     AHNAF.COMPANY_NAME    Q9
3  2024-12-03-20.44.07.528574  AHNAF.MOVIE_COMPANIES    Q5
4  2024-12-03-20.44.07.528574          AHNAF.KEYWORD    Q7
5  2024-12-03-20.44.07.528574        AHNAF.LINK_TYPE    Q6
6  2024-12-03-20.44.07.528574     AHNAF.COMPANY_TYPE    Q8


# extract optimized statement

In [31]:
if not os.path.isfile('db2.ipynb'):
    os.system('wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb')

%run db2.ipynb

<>:1708: SyntaxWarning: invalid escape sequence '\s'
<>:2305: SyntaxWarning: invalid escape sequence '\?'
/tmp/ipykernel_854004/2299624180.py:1708: SyntaxWarning: invalid escape sequence '\s'
  firstCommand = "(?:^\s*)([a-zA-Z]+)(?:\s+.*|$)"
/tmp/ipykernel_854004/2299624180.py:2305: SyntaxWarning: invalid escape sequence '\?'
  pattern = "\?\*[0-9]+"


         Install itables if you want to enable scrolling of result sets.
Db2 Extensions Loaded. Version: 2024-09-16


In [32]:
from dotenv import dotenv_values

In [33]:
db2creds = dotenv_values('db2con.env')
%sql CONNECT CREDENTIALS db2creds

Connection successful. TESTDB @ localhost 


In [34]:
%sql SET CURRENT SCHEMA AHNAF

Command completed.


In [35]:
explain_stmt = %sql select STATEMENT_TEXT from EXPLAIN_STATEMENT where EXPLAIN_TIME = '{query1_ts}' AND EXPLAIN_LEVEL = 'S'

In [36]:
explain_stmt['STATEMENT_TEXT'][0]

'SELECT Q9.NAME AS "COMPANY_NAME", Q6.LINK AS "LINK_TYPE", Q1.TITLE AS "WESTERN_FOLLOW_UP" FROM AHNAF.TITLE AS Q1, AHNAF.MOVIE_LINK AS Q2, AHNAF.MOVIE_KEYWORD AS Q3, AHNAF.MOVIE_INFO AS Q4, AHNAF.MOVIE_COMPANIES AS Q5, AHNAF.LINK_TYPE AS Q6, AHNAF.KEYWORD AS Q7, AHNAF.COMPANY_TYPE AS Q8, AHNAF.COMPANY_NAME AS Q9 WHERE (Q5.MOVIE_ID = Q4.MOVIE_ID) AND (Q3.MOVIE_ID = Q4.MOVIE_ID) AND (Q2.MOVIE_ID = Q4.MOVIE_ID) AND (Q4.MOVIE_ID = Q1.ID) AND (Q5.COMPANY_ID = Q9.ID) AND (Q5.COMPANY_TYPE_ID = Q8.ID) AND (Q3.KEYWORD_ID = Q7.ID) AND (Q6.ID = Q2.LINK_TYPE_ID) AND (Q1.PRODUCTION_YEAR <= 2020) AND (1950 <= Q1.PRODUCTION_YEAR) AND Q5.NOTE IS NULL AND (Q6.LINK LIKE \'%featured in%\') AND (Q7.KEYWORD = \'helicopter\') AND (Q8.KIND = \'distributors\') AND (Q9.COUNTRY_CODE = \'[us]\') AND Q4.INFO IN (\'Sweden               \', \'Norway               \', \'Germany              \', \'Denmark              \', \'Swedish              \', \'Denish               \', \'Norwegian            \', \'German       

In [37]:
import re

# Sample SQL text (multiline)
sql_text = explain_stmt['STATEMENT_TEXT'][0]

# Function to extract the FROM clause, handling multiline SQL
def extract_from_clause(sql_text):
    # Extract the FROM clause (up to WHERE, GROUP BY, etc.), handling newlines and extra spaces
    from_clause_match = re.search(r'\bFROM\b\s+(.+?)\s*(\bWHERE\b|\bGROUP BY\b|\bORDER BY\b|$)', sql_text, re.IGNORECASE | re.DOTALL)
    if from_clause_match:
        from_clause = from_clause_match.group(1).strip()
        return from_clause
    return None

# Function to extract table names and their aliases from the FROM clause
def extract_table_aliases(sql_text):
    # Extract the FROM clause
    from_clause = extract_from_clause(sql_text)
    
    if from_clause:
        print(f"Captured FROM clause: {from_clause}")  # Debugging step to print captured FROM clause

        # Find all table names and aliases in the form "table AS alias" or "table alias"
        table_alias_pairs = re.findall(r'([A-Za-z0-9_\.]+)\s+(?:AS\s+)?([A-Za-z0-9_]+)', from_clause, re.IGNORECASE)

        # Flip the dictionary: aliases as keys, table names as values
        alias_table_dict = {pair[1]: pair[0] for pair in table_alias_pairs}
        return alias_table_dict
    return {}

# Extract table aliases (flipped)
alias_table_dict = extract_table_aliases(sql_text)

# Display the extracted alias-table dictionary
print("Extracted alias-table dictionary:", alias_table_dict)


Captured FROM clause: AHNAF.TITLE AS Q1, AHNAF.MOVIE_LINK AS Q2, AHNAF.MOVIE_KEYWORD AS Q3, AHNAF.MOVIE_INFO AS Q4, AHNAF.MOVIE_COMPANIES AS Q5, AHNAF.LINK_TYPE AS Q6, AHNAF.KEYWORD AS Q7, AHNAF.COMPANY_TYPE AS Q8, AHNAF.COMPANY_NAME AS Q9
Extracted alias-table dictionary: {'Q1': 'AHNAF.TITLE', 'Q2': 'AHNAF.MOVIE_LINK', 'Q3': 'AHNAF.MOVIE_KEYWORD', 'Q4': 'AHNAF.MOVIE_INFO', 'Q5': 'AHNAF.MOVIE_COMPANIES', 'Q6': 'AHNAF.LINK_TYPE', 'Q7': 'AHNAF.KEYWORD', 'Q8': 'AHNAF.COMPANY_TYPE', 'Q9': 'AHNAF.COMPANY_NAME'}


In [38]:
df.head()

,EXPLAIN_TIME,OPERATOR_ID,OPERATOR_TYPE,PREDICATE1,PREDICATE2,PREDICATE3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3,JOIN_KEY,SOURCE_ID,TABLE
0,2024-12-03-20.44.07.528574,1,RETURN,NaN,NaN,NaN,1.000000,1.000000,1.0,NaN,NaN,NaN
1,2024-12-03-20.44.07.528574,2,TQ,NaN,NaN,NaN,1.000000,1.000000,1.0,NaN,NaN,NaN
2,2024-12-03-20.44.07.528574,3,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,Q4.MOVIE_ID = Q1.ID,NaN,NaN
3,2024-12-03-20.44.07.528574,4,TBSCAN,Q1.PRODUCTION_YEAR <= 2020,Q1.PRODUCTION_YEAR >= 1950,None,0.971485,0.894801,1.0,NaN,-1.0,AHNAF.TITLE
4,2024-12-03-20.44.07.528574,5,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,Q5.MOVIE_ID = Q4.MOVIE_ID,NaN,NaN


In [39]:
pd.set_option('display.max_colwidth', None)

In [40]:
import pandas as pd
import re

# # Sample alias-table dictionary extracted from SQL
# alias_table_dict = {
#     'Q1': 'TPCDS.CUSTOMER',
#     'Q2': 'TPCDS.DATE_DIM2'
# }

# Function to replace all aliases in the predicate with the corresponding table names using alias_table_dict
def replace_alias_with_table(predicate, alias_table_dict):
    if isinstance(predicate, str):
        # Replace all aliases in the string with the corresponding table names
        for alias, table_name in alias_table_dict.items():
            predicate = re.sub(fr'\b{alias}\b\.', f'{table_name}.', predicate)
    return predicate

# Apply the replacement to each predicate column
for index, row in df.iterrows():
    df.at[index, 'PREDICATE1'] = replace_alias_with_table(row['PREDICATE1'], alias_table_dict)
    df.at[index, 'PREDICATE2'] = replace_alias_with_table(row['PREDICATE2'], alias_table_dict)
    df.at[index, 'PREDICATE3'] = replace_alias_with_table(row['PREDICATE3'], alias_table_dict)
    df.at[index, 'JOIN_KEY'] = replace_alias_with_table(row['JOIN_KEY'], alias_table_dict)

# Display the updated DataFrame
print(df)


                  EXPLAIN_TIME  OPERATOR_ID OPERATOR_TYPE  \
0   2024-12-03-20.44.07.528574            1        RETURN   
1   2024-12-03-20.44.07.528574            2        TQ       
2   2024-12-03-20.44.07.528574            3        HSJOIN   
3   2024-12-03-20.44.07.528574            4        TBSCAN   
4   2024-12-03-20.44.07.528574            5        HSJOIN   
5   2024-12-03-20.44.07.528574            6        TBSCAN   
6   2024-12-03-20.44.07.528574            7        HSJOIN   
7   2024-12-03-20.44.07.528574            8        TBSCAN   
8   2024-12-03-20.44.07.528574            9        HSJOIN   
9   2024-12-03-20.44.07.528574           10        HSJOIN   
10  2024-12-03-20.44.07.528574           11        TBSCAN   
11  2024-12-03-20.44.07.528574           12        HSJOIN   
12  2024-12-03-20.44.07.528574           13        HSJOIN   
13  2024-12-03-20.44.07.528574           14        TBSCAN   
14  2024-12-03-20.44.07.528574           15        HSJOIN   
15  2024-12-03-20.44.07.

In [41]:
df.columns

Index(['EXPLAIN_TIME', 'OPERATOR_ID', 'OPERATOR_TYPE', 'PREDICATE1',
       'PREDICATE2', 'PREDICATE3', 'FILTER_FACTOR1', 'FILTER_FACTOR2',
       'FILTER_FACTOR3', 'JOIN_KEY', 'SOURCE_ID', 'TABLE'],
      dtype='object')

In [42]:
# Creating new columns for parsed predicates
for i in range(1, 4):
    print(i)
    pred_col = f'PREDICATE{i}'
    parsed_col = f'PARSED_PRED{i}'
    df[parsed_col] = df[pred_col].apply(parse_predicate)

# Display the updated dataframe
df

1
2
3


,EXPLAIN_TIME,OPERATOR_ID,OPERATOR_TYPE,PREDICATE1,PREDICATE2,PREDICATE3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3,JOIN_KEY,SOURCE_ID,TABLE,PARSED_PRED1,PARSED_PRED2,PARSED_PRED3
0,2024-12-03-20.44.07.528574,1,RETURN,NaN,NaN,NaN,1.000000,1.000000,1.0,NaN,NaN,NaN,None,None,None
1,2024-12-03-20.44.07.528574,2,TQ,NaN,NaN,NaN,1.000000,1.000000,1.0,NaN,NaN,NaN,None,None,None
2,2024-12-03-20.44.07.528574,3,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_INFO.MOVIE_ID = AHNAF.TITLE.ID,NaN,NaN,None,None,None
3,2024-12-03-20.44.07.528574,4,TBSCAN,AHNAF.TITLE.PRODUCTION_YEAR <= 2020,AHNAF.TITLE.PRODUCTION_YEAR >= 1950,None,0.971485,0.894801,1.0,NaN,-1.0,AHNAF.TITLE,"(AHNAF.TITLE.PRODUCTION_YEAR, <=, 2020)","(AHNAF.TITLE.PRODUCTION_YEAR, >=, 1950)",None
4,2024-12-03-20.44.07.528574,5,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.MOVIE_ID = AHNAF.MOVIE_INFO.MOVIE_ID,NaN,NaN,None,None,None
5,2024-12-03-20.44.07.528574,6,TBSCAN,"AHNAF.MOVIE_INFO.INFO IN ('Sweden ', 'Norway ', 'Germany ', 'Denmark ', 'Swedish ', 'Denish ', 'Norwegian ', 'German ', 'USA ', 'CANADA ', 'Netherlands ', 'Brazil ', 'UK ', 'Belgium ', 'Finland ', 'Hungary ', 'Estonia ', 'worldwide ', 'Australia ', 'Spain ', 'France ', 'Japan ', 'Columbia ', 'Slovenia ', 'Israel ', 'Venezuela ', 'Nigeria ', 'Philippines ', 'New Zealand ', 'Ireland ', 'Romania ', 'Non-USA ', 'Bulgaria ', 'Argentina ', 'Malaysia ', 'Singapore ', 'Turkey ', 'Sri Lanka ', 'Italy ', 'Indonesia ', 'South Korea ', 'Vietnam ', 'Slovakia ', 'Czech Republic ', 'China ', 'Portugal ', 'Greece ', 'Republic of Macedonia', 'Serbia ', 'Jamaica ', 'Switzerland ', 'Yugoslavia ', 'Mexico ', 'Austria ', 'Russia ')",None,None,0.029517,1.000000,1.0,NaN,-1.0,AHNAF.MOVIE_INFO,"(AHNAF.MOVIE_INFO.INFO, IN, [Sweden, Norway, Germany, Denmark, Swedish, Denish, Norwegian, German, USA, CANADA, Netherlands, Brazil, UK, Belgium, Finland, Hungary, Estonia, worldwide, Australia, Spain, France, Japan, Columbia, Slovenia, Israel, Venezuela, Nigeria, Philippines, New Zealand, Ireland, Romania, Non-USA, Bulgaria, Argentina, Malaysia, Singapore, Turkey, Sri Lanka, Italy, Indonesia, South Korea, Vietnam, Slovakia, Czech Republic, China, Portugal, Greece, Republic of Macedonia, Serbia, Jamaica, Switzerland, Yugoslavia, Mexico, Austria, Russia])",None,None
6,2024-12-03-20.44.07.528574,7,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.COMPANY_ID = AHNAF.COMPANY_NAME.ID,NaN,NaN,None,None,None
7,2024-12-03-20.44.07.528574,8,TBSCAN,AHNAF.COMPANY_NAME.COUNTRY_CODE = '[us]',None,None,0.359728,1.000000,1.0,NaN,-1.0,AHNAF.COMPANY_NAME,"(AHNAF.COMPANY_NAME.COUNTRY_CODE, =, '[us]')",None,None
8,2024-12-03-20.44.07.528574,9,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.COMPANY_TYPE_ID = AHNAF.COMPANY_TYPE.ID,NaN,NaN,None,None,None
9,2024-12-03-20.44.07.528574,10,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_LINK.MOVIE_ID = AHNAF.MOVIE_COMPANIES.MOVIE_ID,NaN,NaN,None,None,None


# Calculating Tree Node heights

In [43]:
# Read the CSV file
file_path = f'{base_dir}/EXPLAIN_STREAM.csv'
df_stream = pd.read_csv(file_path)

# Filter for the given EXPLAIN_TIME value and non-NaN OBJECT_NAME
df_stream = df_stream.loc[(df_stream['EXPLAIN_TIME'] == query1_ts)].copy()

In [44]:
df_stream.columns

Index(['EXPLAIN_REQUESTER', 'EXPLAIN_TIME', 'SOURCE_NAME', 'SOURCE_SCHEMA',
       'SOURCE_VERSION', 'EXPLAIN_LEVEL', 'STMTNO', 'SECTNO', 'STREAM_ID',
       'SOURCE_TYPE', 'SOURCE_ID', 'TARGET_TYPE', 'TARGET_ID', 'OBJECT_SCHEMA',
       'OBJECT_NAME', 'STREAM_COUNT', 'COLUMN_COUNT', 'PREDICATE_ID',
       'COLUMN_NAMES', 'PMID', 'SINGLE_NODE', 'PARTITION_COLUMNS',
       'SEQUENCE_SIZES', 'OBJECT_TENANTID'],
      dtype='object')

In [45]:
df_stream = df_stream[['SOURCE_ID', 'TARGET_ID']]

In [46]:
df_stream.head()

,SOURCE_ID,TARGET_ID
6388,-1,4
6389,4,3
6390,-1,6
6391,6,5
6392,-1,8


In [47]:
df_stream

,SOURCE_ID,TARGET_ID
6388,-1,4
6389,4,3
6390,-1,6
6391,6,5
6392,-1,8
6393,8,7
6394,-1,11
6395,11,10
6396,-1,14
6397,14,13


In [48]:

import pandas as pd
import numpy as np

# Step 1: Generate the adjacency matrix (from previous code)
# data = {
#     'SOURCE_ID': [-1, 3, -1, 4, 2],
#     'TARGET_ID': [3, 2, 4, 2, 1]
# }
# df = pd.DataFrame(data)

# Extract all unique nodes from SOURCE_ID and TARGET_ID
node_ids = set(df_stream['SOURCE_ID']).union(set(df_stream['TARGET_ID'])) - {-1}
node_ids = sorted(node_ids)

# Map node ids to indices for the adjacency matrix
node_to_index = {node: idx for idx, node in enumerate(node_ids)}

# Initialize the adjacency matrix with zeros
n = len(node_ids)
adj_matrix = np.zeros((n, n), dtype=int)

# Fill the adjacency matrix based on the parent-child relationships
for _, row in df_stream.iterrows():
    source = row['SOURCE_ID']
    target = row['TARGET_ID']
    
    if source != -1:
        parent_idx = node_to_index[target]
        child_idx = node_to_index[source]
        adj_matrix[parent_idx][child_idx] = 1

# Convert to DataFrame for better display
adj_matrix_df = pd.DataFrame(adj_matrix, index=node_ids, columns=node_ids)
print("Adjacency Matrix:")
print(adj_matrix_df)


Adjacency Matrix:
    1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18  19
1    0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
2    0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
3    0   0   0   1   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
4    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
5    0   0   0   0   0   1   1   0   0   0   0   0   0   0   0   0   0   0   0
6    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
7    0   0   0   0   0   0   0   1   1   0   0   0   0   0   0   0   0   0   0
8    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
9    0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   1
10   0   0   0   0   0   0   0   0   0   0   1   1   0   0   0   0   0   0   0
11   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
12   0   0   0   0   0   0   0   0

In [49]:
# Reset the index of the adjacency matrix to be 0-based
adj_matrix_df.index = adj_matrix_df.index - 1  # Convert row indices to 0-based
adj_matrix_df.columns = adj_matrix_df.columns - 1  # Convert column indices to 0-based

print("Adjacency Matrix with 0-Based Index:")
print(adj_matrix_df)

Adjacency Matrix with 0-Based Index:
    0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18
0    0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
1    0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
2    0   0   0   1   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
3    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
4    0   0   0   0   0   1   1   0   0   0   0   0   0   0   0   0   0   0   0
5    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
6    0   0   0   0   0   0   0   1   1   0   0   0   0   0   0   0   0   0   0
7    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
8    0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   1
9    0   0   0   0   0   0   0   0   0   0   1   1   0   0   0   0   0   0   0
10   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
11   0   0   0 

# Find the BFS sequence

In [50]:
import numpy as np
from collections import deque

def bfs_from_adj_matrix(adj_matrix, start_node):
    """
    Perform BFS on a graph represented by an adjacency matrix.

    Args:
        adj_matrix (np.ndarray): Adjacency matrix of the graph.
        start_node (int): Starting node for BFS.

    Returns:
        list: List of nodes in the order they are visited.
    """
    num_nodes = adj_matrix.shape[0]  # Number of nodes
    visited = [False] * num_nodes    # Track visited nodes
    queue = deque([start_node])      # Initialize queue with the start node
    visited[start_node] = True       # Mark the start node as visited

    bfs_order = []  # List to store BFS traversal order

    while queue:
        node = queue.popleft()       # Dequeue the next node
        bfs_order.append(node)       # Record the visited node

        # Check all neighbors (columns in the row of this node)
        for neighbor, is_connected in enumerate(adj_matrix[node]):
            if is_connected and not visited[neighbor]:  # If connected and not visited
                queue.append(neighbor)                  # Enqueue the neighbor
                visited[neighbor] = True                # Mark as visited

    return bfs_order

# Convert adj_matrix_df to NumPy array
adj_matrix_np = adj_matrix_df.values

# Perform BFS starting from node 0
bfs_order = bfs_from_adj_matrix(adj_matrix_np, start_node=0)
print("BFS Row Index Order:", bfs_order)


BFS Row Index Order: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 18, 10, 11, 12, 17, 13, 14, 15, 16]


In [51]:
# Convert 0-based indexed adjacency matrix to adj_list
adj_list = []

# Iterate over the adjacency matrix
for parent_idx, row in enumerate(adj_matrix):  # parent_idx is the row index
    for child_idx, is_child in enumerate(row):  # child_idx is the column index
        if is_child:  # If there's a connection (value is 1)
            adj_list.append((parent_idx, child_idx))  # Use 0-based indices directly

adj_list_tensor = torch.LongTensor(np.array(adj_list))

print("Adjacency List:")
print(adj_list_tensor)


Adjacency List:
tensor([[ 0,  1],
        [ 1,  2],
        [ 2,  3],
        [ 2,  4],
        [ 4,  5],
        [ 4,  6],
        [ 6,  7],
        [ 6,  8],
        [ 8,  9],
        [ 8, 18],
        [ 9, 10],
        [ 9, 11],
        [11, 12],
        [11, 17],
        [12, 13],
        [12, 14],
        [14, 15],
        [14, 16]])


In [52]:
# Step 1: Perform BFS to get the order of nodes
bfs_order = bfs_from_adj_matrix(adj_matrix, start_node=0)  # Adjust start_node as needed

# Step 2: Create a mapping from BFS order to indices for sorting
bfs_index_map = {node: i for i, node in enumerate(bfs_order)}

# Step 3: Reorder adj_list based on BFS order
adj_list = [
    (parent, child)
    for parent in bfs_order
    for child, is_child in enumerate(adj_matrix[parent])
    if is_child  # Keep only valid edges
]

# Step 4: Convert to tensor if needed
adj_list_tensor = torch.LongTensor(np.array(adj_list))

print("Reordered Adjacency List:")
print(adj_list_tensor)

Reordered Adjacency List:
tensor([[ 0,  1],
        [ 1,  2],
        [ 2,  3],
        [ 2,  4],
        [ 4,  5],
        [ 4,  6],
        [ 6,  7],
        [ 6,  8],
        [ 8,  9],
        [ 8, 18],
        [ 9, 10],
        [ 9, 11],
        [11, 12],
        [11, 17],
        [12, 13],
        [12, 14],
        [14, 15],
        [14, 16]])


In [53]:
edge_index = adj_list_tensor.t()

In [54]:
edge_index

tensor([[ 0,  1,  2,  2,  4,  4,  6,  6,  8,  8,  9,  9, 11, 11, 12, 12, 14, 14],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 18, 10, 11, 12, 17, 13, 14, 15, 16]])

In [55]:
from model.database_util import floyd_warshall_rewrite

In [56]:
df

,EXPLAIN_TIME,OPERATOR_ID,OPERATOR_TYPE,PREDICATE1,PREDICATE2,PREDICATE3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3,JOIN_KEY,SOURCE_ID,TABLE,PARSED_PRED1,PARSED_PRED2,PARSED_PRED3
0,2024-12-03-20.44.07.528574,1,RETURN,NaN,NaN,NaN,1.000000,1.000000,1.0,NaN,NaN,NaN,None,None,None
1,2024-12-03-20.44.07.528574,2,TQ,NaN,NaN,NaN,1.000000,1.000000,1.0,NaN,NaN,NaN,None,None,None
2,2024-12-03-20.44.07.528574,3,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_INFO.MOVIE_ID = AHNAF.TITLE.ID,NaN,NaN,None,None,None
3,2024-12-03-20.44.07.528574,4,TBSCAN,AHNAF.TITLE.PRODUCTION_YEAR <= 2020,AHNAF.TITLE.PRODUCTION_YEAR >= 1950,None,0.971485,0.894801,1.0,NaN,-1.0,AHNAF.TITLE,"(AHNAF.TITLE.PRODUCTION_YEAR, <=, 2020)","(AHNAF.TITLE.PRODUCTION_YEAR, >=, 1950)",None
4,2024-12-03-20.44.07.528574,5,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.MOVIE_ID = AHNAF.MOVIE_INFO.MOVIE_ID,NaN,NaN,None,None,None
5,2024-12-03-20.44.07.528574,6,TBSCAN,"AHNAF.MOVIE_INFO.INFO IN ('Sweden ', 'Norway ', 'Germany ', 'Denmark ', 'Swedish ', 'Denish ', 'Norwegian ', 'German ', 'USA ', 'CANADA ', 'Netherlands ', 'Brazil ', 'UK ', 'Belgium ', 'Finland ', 'Hungary ', 'Estonia ', 'worldwide ', 'Australia ', 'Spain ', 'France ', 'Japan ', 'Columbia ', 'Slovenia ', 'Israel ', 'Venezuela ', 'Nigeria ', 'Philippines ', 'New Zealand ', 'Ireland ', 'Romania ', 'Non-USA ', 'Bulgaria ', 'Argentina ', 'Malaysia ', 'Singapore ', 'Turkey ', 'Sri Lanka ', 'Italy ', 'Indonesia ', 'South Korea ', 'Vietnam ', 'Slovakia ', 'Czech Republic ', 'China ', 'Portugal ', 'Greece ', 'Republic of Macedonia', 'Serbia ', 'Jamaica ', 'Switzerland ', 'Yugoslavia ', 'Mexico ', 'Austria ', 'Russia ')",None,None,0.029517,1.000000,1.0,NaN,-1.0,AHNAF.MOVIE_INFO,"(AHNAF.MOVIE_INFO.INFO, IN, [Sweden, Norway, Germany, Denmark, Swedish, Denish, Norwegian, German, USA, CANADA, Netherlands, Brazil, UK, Belgium, Finland, Hungary, Estonia, worldwide, Australia, Spain, France, Japan, Columbia, Slovenia, Israel, Venezuela, Nigeria, Philippines, New Zealand, Ireland, Romania, Non-USA, Bulgaria, Argentina, Malaysia, Singapore, Turkey, Sri Lanka, Italy, Indonesia, South Korea, Vietnam, Slovakia, Czech Republic, China, Portugal, Greece, Republic of Macedonia, Serbia, Jamaica, Switzerland, Yugoslavia, Mexico, Austria, Russia])",None,None
6,2024-12-03-20.44.07.528574,7,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.COMPANY_ID = AHNAF.COMPANY_NAME.ID,NaN,NaN,None,None,None
7,2024-12-03-20.44.07.528574,8,TBSCAN,AHNAF.COMPANY_NAME.COUNTRY_CODE = '[us]',None,None,0.359728,1.000000,1.0,NaN,-1.0,AHNAF.COMPANY_NAME,"(AHNAF.COMPANY_NAME.COUNTRY_CODE, =, '[us]')",None,None
8,2024-12-03-20.44.07.528574,9,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.COMPANY_TYPE_ID = AHNAF.COMPANY_TYPE.ID,NaN,NaN,None,None,None
9,2024-12-03-20.44.07.528574,10,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_LINK.MOVIE_ID = AHNAF.MOVIE_COMPANIES.MOVIE_ID,NaN,NaN,None,None,None


In [57]:
bfs_order

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 18, 10, 11, 12, 17, 13, 14, 15, 16]

In [58]:
# Reorder the DataFrame rows based on BFS order
df = df.iloc[bfs_order].reset_index(drop=True)

df

,EXPLAIN_TIME,OPERATOR_ID,OPERATOR_TYPE,PREDICATE1,PREDICATE2,PREDICATE3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3,JOIN_KEY,SOURCE_ID,TABLE,PARSED_PRED1,PARSED_PRED2,PARSED_PRED3
0,2024-12-03-20.44.07.528574,1,RETURN,NaN,NaN,NaN,1.000000,1.000000,1.0,NaN,NaN,NaN,None,None,None
1,2024-12-03-20.44.07.528574,2,TQ,NaN,NaN,NaN,1.000000,1.000000,1.0,NaN,NaN,NaN,None,None,None
2,2024-12-03-20.44.07.528574,3,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_INFO.MOVIE_ID = AHNAF.TITLE.ID,NaN,NaN,None,None,None
3,2024-12-03-20.44.07.528574,4,TBSCAN,AHNAF.TITLE.PRODUCTION_YEAR <= 2020,AHNAF.TITLE.PRODUCTION_YEAR >= 1950,None,0.971485,0.894801,1.0,NaN,-1.0,AHNAF.TITLE,"(AHNAF.TITLE.PRODUCTION_YEAR, <=, 2020)","(AHNAF.TITLE.PRODUCTION_YEAR, >=, 1950)",None
4,2024-12-03-20.44.07.528574,5,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.MOVIE_ID = AHNAF.MOVIE_INFO.MOVIE_ID,NaN,NaN,None,None,None
5,2024-12-03-20.44.07.528574,6,TBSCAN,"AHNAF.MOVIE_INFO.INFO IN ('Sweden ', 'Norway ', 'Germany ', 'Denmark ', 'Swedish ', 'Denish ', 'Norwegian ', 'German ', 'USA ', 'CANADA ', 'Netherlands ', 'Brazil ', 'UK ', 'Belgium ', 'Finland ', 'Hungary ', 'Estonia ', 'worldwide ', 'Australia ', 'Spain ', 'France ', 'Japan ', 'Columbia ', 'Slovenia ', 'Israel ', 'Venezuela ', 'Nigeria ', 'Philippines ', 'New Zealand ', 'Ireland ', 'Romania ', 'Non-USA ', 'Bulgaria ', 'Argentina ', 'Malaysia ', 'Singapore ', 'Turkey ', 'Sri Lanka ', 'Italy ', 'Indonesia ', 'South Korea ', 'Vietnam ', 'Slovakia ', 'Czech Republic ', 'China ', 'Portugal ', 'Greece ', 'Republic of Macedonia', 'Serbia ', 'Jamaica ', 'Switzerland ', 'Yugoslavia ', 'Mexico ', 'Austria ', 'Russia ')",None,None,0.029517,1.000000,1.0,NaN,-1.0,AHNAF.MOVIE_INFO,"(AHNAF.MOVIE_INFO.INFO, IN, [Sweden, Norway, Germany, Denmark, Swedish, Denish, Norwegian, German, USA, CANADA, Netherlands, Brazil, UK, Belgium, Finland, Hungary, Estonia, worldwide, Australia, Spain, France, Japan, Columbia, Slovenia, Israel, Venezuela, Nigeria, Philippines, New Zealand, Ireland, Romania, Non-USA, Bulgaria, Argentina, Malaysia, Singapore, Turkey, Sri Lanka, Italy, Indonesia, South Korea, Vietnam, Slovakia, Czech Republic, China, Portugal, Greece, Republic of Macedonia, Serbia, Jamaica, Switzerland, Yugoslavia, Mexico, Austria, Russia])",None,None
6,2024-12-03-20.44.07.528574,7,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.COMPANY_ID = AHNAF.COMPANY_NAME.ID,NaN,NaN,None,None,None
7,2024-12-03-20.44.07.528574,8,TBSCAN,AHNAF.COMPANY_NAME.COUNTRY_CODE = '[us]',None,None,0.359728,1.000000,1.0,NaN,-1.0,AHNAF.COMPANY_NAME,"(AHNAF.COMPANY_NAME.COUNTRY_CODE, =, '[us]')",None,None
8,2024-12-03-20.44.07.528574,9,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.COMPANY_TYPE_ID = AHNAF.COMPANY_TYPE.ID,NaN,NaN,None,None,None
9,2024-12-03-20.44.07.528574,10,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_LINK.MOVIE_ID = AHNAF.MOVIE_COMPANIES.MOVIE_ID,NaN,NaN,None,None,None


In [59]:
df.shape
N = df.shape[0]

In [60]:
if len(edge_index) == 0:
            shortest_path_result = np.array([[0]])
            path = np.array([[0]])
            adj = torch.tensor([[0]]).bool()
else:
            adj = torch.zeros([N,N], dtype=torch.bool)
            adj[edge_index[0,:], edge_index[1,:]] = True
            
            shortest_path_result = floyd_warshall_rewrite(adj.numpy())
        
rel_pos = torch.from_numpy((shortest_path_result)).long()

In [61]:
rel_pos

tensor([[ 0,  1,  2,  3,  3,  4,  4,  5,  5,  6,  7,  7,  8,  9,  9, 10, 10,  8,
          6],
        [60,  0,  1,  2,  2,  3,  3,  4,  4,  5,  6,  6,  7,  8,  8,  9,  9,  7,
          5],
        [60, 60,  0,  1,  1,  2,  2,  3,  3,  4,  5,  5,  6,  7,  7,  8,  8,  6,
          4],
        [60, 60, 60,  0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
         60],
        [60, 60, 60, 60,  0,  1,  1,  2,  2,  3,  4,  4,  5,  6,  6,  7,  7,  5,
          3],
        [60, 60, 60, 60, 60,  0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
         60],
        [60, 60, 60, 60, 60, 60,  0,  1,  1,  2,  3,  3,  4,  5,  5,  6,  6,  4,
          2],
        [60, 60, 60, 60, 60, 60, 60,  0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
         60],
        [60, 60, 60, 60, 60, 60, 60, 60,  0,  1,  2,  2,  3,  4,  4,  5,  5,  3,
          1],
        [60, 60, 60, 60, 60, 60, 60, 60, 60,  0,  1,  1,  2,  3,  3,  4,  4,  2,
         60],
        [60, 60, 60, 60, 60, 60, 60, 60, 60, 60,  

In [62]:
def pad_attn_bias_unsqueeze(x, padlen):
    xlen = x.size(0)
    if xlen < padlen:
        new_x = x.new_zeros([padlen, padlen], dtype=x.dtype).fill_(float('-inf'))
        new_x[:xlen, :xlen] = x
        new_x[xlen:, :xlen] = 0
        x = new_x
    return x.unsqueeze(0)

In [63]:
attn_bias = torch.zeros([N+1,N+1], dtype=torch.float)
attn_bias[1:, 1:][rel_pos >= rel_pos_max] = float('-inf')

In [64]:
attn_bias = pad_attn_bias_unsqueeze(attn_bias, max_node + 1)

In [65]:
def pad_rel_pos_unsqueeze(x, padlen):
    x = x + 1
    xlen = x.size(0)
    if xlen < padlen:
        new_x = x.new_zeros([padlen, padlen], dtype=x.dtype)
        new_x[:xlen, :xlen] = x
        x = new_x
    return x.unsqueeze(0)

In [66]:
rel_pos.shape

torch.Size([19, 19])

In [67]:
rel_pos

tensor([[ 0,  1,  2,  3,  3,  4,  4,  5,  5,  6,  7,  7,  8,  9,  9, 10, 10,  8,
          6],
        [60,  0,  1,  2,  2,  3,  3,  4,  4,  5,  6,  6,  7,  8,  8,  9,  9,  7,
          5],
        [60, 60,  0,  1,  1,  2,  2,  3,  3,  4,  5,  5,  6,  7,  7,  8,  8,  6,
          4],
        [60, 60, 60,  0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
         60],
        [60, 60, 60, 60,  0,  1,  1,  2,  2,  3,  4,  4,  5,  6,  6,  7,  7,  5,
          3],
        [60, 60, 60, 60, 60,  0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
         60],
        [60, 60, 60, 60, 60, 60,  0,  1,  1,  2,  3,  3,  4,  5,  5,  6,  6,  4,
          2],
        [60, 60, 60, 60, 60, 60, 60,  0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
         60],
        [60, 60, 60, 60, 60, 60, 60, 60,  0,  1,  2,  2,  3,  4,  4,  5,  5,  3,
          1],
        [60, 60, 60, 60, 60, 60, 60, 60, 60,  0,  1,  1,  2,  3,  3,  4,  4,  2,
         60],
        [60, 60, 60, 60, 60, 60, 60, 60, 60, 60,  

# creating attn_bias tensor

In [68]:
attn_bias = torch.zeros([N+1,N+1], dtype=torch.float)
attn_bias[1:, 1:][rel_pos >= rel_pos_max] = float('-inf')
#attn_bias = pad_attn_bias_unsqueeze(attn_bias, max_node + 1)

In [69]:
attn_bias = pad_attn_bias_unsqueeze(attn_bias, max_node + 1)
rel_pos = pad_rel_pos_unsqueeze(rel_pos, max_node)

In [70]:
attn_bias.shape

torch.Size([1, 31, 31])

In [71]:
rel_pos.shape

torch.Size([1, 30, 30])

# Computing longest path for each node

In [72]:
# Step 6: Define a function to compute the longest path from a node to a leaf
def longest_path_from_node(node_idx, adj_matrix, memo):
    # If this node's longest path is already computed, return the cached result
    if node_idx in memo:
        return memo[node_idx]
    
    # If the node is a leaf node (no outgoing edges)
    if np.sum(adj_matrix[node_idx]) == 0:
        memo[node_idx] = 0  # Longest path from a leaf is 0
        return 0
    
    # Otherwise, recursively calculate the longest path from this node to a leaf
    max_length = 0
    for child_idx in range(len(adj_matrix)):
        if adj_matrix[node_idx][child_idx] == 1:  # There's a connection to this child
            max_length = max(max_length, 1 + longest_path_from_node(child_idx, adj_matrix, memo))
    
    memo[node_idx] = max_length  # Memoize the result
    return memo[node_idx]

memo = {}
longest_paths = {}

for node in node_ids:
    node_idx = node_to_index[node]
    longest_paths[node] = longest_path_from_node(node_idx, adj_matrix, memo) + 1

# Step 8: Display the longest path for each node
print("\nLongest Path from Each Node to a Leaf:")
for node, length in longest_paths.items():
    print(f"Node {node}: Longest path = {length}")


Longest Path from Each Node to a Leaf:
Node 1: Longest path = 11
Node 2: Longest path = 10
Node 3: Longest path = 9
Node 4: Longest path = 1
Node 5: Longest path = 8
Node 6: Longest path = 1
Node 7: Longest path = 7
Node 8: Longest path = 1
Node 9: Longest path = 6
Node 10: Longest path = 5
Node 11: Longest path = 1
Node 12: Longest path = 4
Node 13: Longest path = 3
Node 14: Longest path = 1
Node 15: Longest path = 2
Node 16: Longest path = 1
Node 17: Longest path = 1
Node 18: Longest path = 1
Node 19: Longest path = 1


In [73]:
type(longest_paths)

dict

In [74]:
longest_paths.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [75]:
# Add a new column 'height' based on the matching OPERATOR_ID
df['HEIGHT'] = df['OPERATOR_ID'].map(longest_paths)

In [76]:
df.shape

(19, 16)

In [77]:
df

,EXPLAIN_TIME,OPERATOR_ID,OPERATOR_TYPE,PREDICATE1,PREDICATE2,PREDICATE3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3,JOIN_KEY,SOURCE_ID,TABLE,PARSED_PRED1,PARSED_PRED2,PARSED_PRED3,HEIGHT
0,2024-12-03-20.44.07.528574,1,RETURN,NaN,NaN,NaN,1.000000,1.000000,1.0,NaN,NaN,NaN,None,None,None,11
1,2024-12-03-20.44.07.528574,2,TQ,NaN,NaN,NaN,1.000000,1.000000,1.0,NaN,NaN,NaN,None,None,None,10
2,2024-12-03-20.44.07.528574,3,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_INFO.MOVIE_ID = AHNAF.TITLE.ID,NaN,NaN,None,None,None,9
3,2024-12-03-20.44.07.528574,4,TBSCAN,AHNAF.TITLE.PRODUCTION_YEAR <= 2020,AHNAF.TITLE.PRODUCTION_YEAR >= 1950,None,0.971485,0.894801,1.0,NaN,-1.0,AHNAF.TITLE,"(AHNAF.TITLE.PRODUCTION_YEAR, <=, 2020)","(AHNAF.TITLE.PRODUCTION_YEAR, >=, 1950)",None,1
4,2024-12-03-20.44.07.528574,5,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.MOVIE_ID = AHNAF.MOVIE_INFO.MOVIE_ID,NaN,NaN,None,None,None,8
5,2024-12-03-20.44.07.528574,6,TBSCAN,"AHNAF.MOVIE_INFO.INFO IN ('Sweden ', 'Norway ', 'Germany ', 'Denmark ', 'Swedish ', 'Denish ', 'Norwegian ', 'German ', 'USA ', 'CANADA ', 'Netherlands ', 'Brazil ', 'UK ', 'Belgium ', 'Finland ', 'Hungary ', 'Estonia ', 'worldwide ', 'Australia ', 'Spain ', 'France ', 'Japan ', 'Columbia ', 'Slovenia ', 'Israel ', 'Venezuela ', 'Nigeria ', 'Philippines ', 'New Zealand ', 'Ireland ', 'Romania ', 'Non-USA ', 'Bulgaria ', 'Argentina ', 'Malaysia ', 'Singapore ', 'Turkey ', 'Sri Lanka ', 'Italy ', 'Indonesia ', 'South Korea ', 'Vietnam ', 'Slovakia ', 'Czech Republic ', 'China ', 'Portugal ', 'Greece ', 'Republic of Macedonia', 'Serbia ', 'Jamaica ', 'Switzerland ', 'Yugoslavia ', 'Mexico ', 'Austria ', 'Russia ')",None,None,0.029517,1.000000,1.0,NaN,-1.0,AHNAF.MOVIE_INFO,"(AHNAF.MOVIE_INFO.INFO, IN, [Sweden, Norway, Germany, Denmark, Swedish, Denish, Norwegian, German, USA, CANADA, Netherlands, Brazil, UK, Belgium, Finland, Hungary, Estonia, worldwide, Australia, Spain, France, Japan, Columbia, Slovenia, Israel, Venezuela, Nigeria, Philippines, New Zealand, Ireland, Romania, Non-USA, Bulgaria, Argentina, Malaysia, Singapore, Turkey, Sri Lanka, Italy, Indonesia, South Korea, Vietnam, Slovakia, Czech Republic, China, Portugal, Greece, Republic of Macedonia, Serbia, Jamaica, Switzerland, Yugoslavia, Mexico, Austria, Russia])",None,None,1
6,2024-12-03-20.44.07.528574,7,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.COMPANY_ID = AHNAF.COMPANY_NAME.ID,NaN,NaN,None,None,None,7
7,2024-12-03-20.44.07.528574,8,TBSCAN,AHNAF.COMPANY_NAME.COUNTRY_CODE = '[us]',None,None,0.359728,1.000000,1.0,NaN,-1.0,AHNAF.COMPANY_NAME,"(AHNAF.COMPANY_NAME.COUNTRY_CODE, =, '[us]')",None,None,1
8,2024-12-03-20.44.07.528574,9,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.COMPANY_TYPE_ID = AHNAF.COMPANY_TYPE.ID,NaN,NaN,None,None,None,6
9,2024-12-03-20.44.07.528574,10,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_LINK.MOVIE_ID = AHNAF.MOVIE_COMPANIES.MOVIE_ID,NaN,NaN,None,None,None,5


# Creating a tensor of heights

In [78]:
heights = df['HEIGHT'].values

In [79]:
heights

array([11, 10,  9,  1,  8,  1,  7,  1,  6,  5,  1,  1,  4,  3,  1,  1,  2,
        1,  1])

In [80]:
torch.LongTensor(heights).size()

torch.Size([19])

In [81]:
heights = pad_1d_unsqueeze(torch.LongTensor(heights), max_node)


In [82]:
heights

tensor([[12, 11, 10,  2,  9,  2,  8,  2,  7,  6,  2,  2,  5,  4,  2,  2,  3,  2,
          2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [83]:
heights.shape

torch.Size([1, 30])

In [84]:
N

19

In [85]:
heights

tensor([[12, 11, 10,  2,  9,  2,  8,  2,  7,  6,  2,  2,  5,  4,  2,  2,  3,  2,
          2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

# Look up the `SORT_SHRHEAP_TOP`

In [86]:
### Step 1: look up the `APPL_ID`, 'ACTIVITY_ID', 'UOW_ID'
# Define the file path and the timestamp value
file_path = f'{base_dir}/success.csv'  # Replace with your CSV file path

# Read the CSV file, selecting only relevant columns
column_headers = ['QUERYID', 'APPL_ID', 'UOW_ID', 'ACTIVITY_ID', 'EXPLAIN_TIME', 'SORT_SHRHEAP_TOP', 'QUERY']
df_success = pd.read_csv(file_path, header=None)

df_success.columns = column_headers

In [87]:
df_success.shape

(2332, 7)

In [88]:
df_success.head(1)

,QUERYID,APPL_ID,UOW_ID,ACTIVITY_ID,EXPLAIN_TIME,SORT_SHRHEAP_TOP,QUERY
0,1,*LOCAL.db2inst1.241204044349,4,1,2024-12-03-20.43.44.200877,66844,"SELECT cn.name AS company_name, lt.link AS link_type,t.title AS western_follow_up FROM company_name AS cn, company_type AS ct, keyword AS k, link_type AS lt, movie_companies AS mc, movie_info AS mi, movie_keyword AS mk, movie_link AS ml, title AS t WHERE cn.country_code ='[us]' AND ct.kind ='distributors' AND k.keyword ='number-in-title' AND lt.link LIKE '%referenced in%' AND mc.note IS NULL AND mi.info IN ('Sweden', 'Norway', 'Germany', 'Denmark', 'Swedish', 'Denish', 'Norwegian', 'German', 'USA', 'CANADA', 'Netherlands', 'Brazil', 'UK', 'Belgium', 'Finland', 'Hungary', 'Estonia', 'worldwide', 'Australia', 'Spain', 'France', 'Germany', 'Japan', 'Hungary', 'Sweden', 'Columbia', 'Slovenia', 'Israel', 'Venezuela', 'Finland', 'Nigeria', 'Philippines', 'New Zealand', 'Ireland', 'Romania', 'Non-USA', 'Bulgaria', 'Argentina', 'Malaysia', 'Singapore', 'Turkey', 'Sri Lanka', 'Italy', 'Indonesia', 'South Korea', 'Vietnam', 'Slovakia', 'Czech Republic', 'China', 'Portugal', 'Greece', 'Republic of Macedonia', 'Serbia', 'Jamaica', 'Switzerland', 'Yugoslavia', 'Mexico', 'Austria', 'Russia') AND t.production_year BETWEEN 1950 AND 2020 AND lt.id = ml.link_type_id AND ml.movie_id = t.id AND t.id = mk.movie_id AND mk.keyword_id = k.id AND t.id = mc.movie_id AND mc.company_type_id = ct.id AND mc.company_id = cn.id AND mi.movie_id = t.id AND ml.movie_id = mk.movie_id AND ml.movie_id = mc.movie_id AND mk.movie_id = mc.movie_id AND ml.movie_id = mi.movie_id AND mk.movie_id = mi.movie_id AND mc.movie_id = mi.movie_id;"


In [89]:
df_query1 = df_success[df_success['EXPLAIN_TIME'] == query1_ts]

In [90]:
df_query1

,QUERYID,APPL_ID,UOW_ID,ACTIVITY_ID,EXPLAIN_TIME,SORT_SHRHEAP_TOP,QUERY
14,15,*LOCAL.db2inst1.241204044349,116,1,2024-12-03-20.44.07.528574,66821,"SELECT cn.name AS company_name, lt.link AS link_type,t.title AS western_follow_up FROM company_name AS cn, company_type AS ct, keyword AS k, link_type AS lt, movie_companies AS mc, movie_info AS mi, movie_keyword AS mk, movie_link AS ml, title AS t WHERE cn.country_code ='[us]' AND ct.kind ='distributors' AND k.keyword ='helicopter' AND lt.link LIKE '%featured in%' AND mc.note IS NULL AND mi.info IN ('Sweden', 'Norway', 'Germany', 'Denmark', 'Swedish', 'Denish', 'Norwegian', 'German', 'USA', 'CANADA', 'Netherlands', 'Brazil', 'UK', 'Belgium', 'Finland', 'Hungary', 'Estonia', 'worldwide', 'Australia', 'Spain', 'France', 'Germany', 'Japan', 'Hungary', 'Sweden', 'Columbia', 'Slovenia', 'Israel', 'Venezuela', 'Finland', 'Nigeria', 'Philippines', 'New Zealand', 'Ireland', 'Romania', 'Non-USA', 'Bulgaria', 'Argentina', 'Malaysia', 'Singapore', 'Turkey', 'Sri Lanka', 'Italy', 'Indonesia', 'South Korea', 'Vietnam', 'Slovakia', 'Czech Republic', 'China', 'Portugal', 'Greece', 'Republic of Macedonia', 'Serbia', 'Jamaica', 'Switzerland', 'Yugoslavia', 'Mexico', 'Austria', 'Russia') AND t.production_year BETWEEN 1950 AND 2020 AND lt.id = ml.link_type_id AND ml.movie_id = t.id AND t.id = mk.movie_id AND mk.keyword_id = k.id AND t.id = mc.movie_id AND mc.company_type_id = ct.id AND mc.company_id = cn.id AND mi.movie_id = t.id AND ml.movie_id = mk.movie_id AND ml.movie_id = mc.movie_id AND mk.movie_id = mc.movie_id AND ml.movie_id = mi.movie_id AND mk.movie_id = mi.movie_id AND mc.movie_id = mi.movie_id;"


In [91]:
df_query1.shape

(1, 7)

## List the final set of dataframes, which I will convert to tensors

In [92]:
# get the SORT_SHRHEAP_TOP
raw_costs = df_query1['SORT_SHRHEAP_TOP'].values.tolist()
QUERYID = df_query1['QUERYID'].values[0]

print(raw_costs)

[66821]


In [93]:
type(raw_costs)

list

In [94]:
import torch
from model.util import Normalizer

# cost_norm = Normalizer(1, 100)
# cost_norm = Normalizer(-3.61192, 12.290855)
#cost_norm = Normalizer(5, 2611)
cost_norm = Normalizer(8.26, 11.12)
cost_labels = torch.from_numpy(cost_norm.normalize_labels(raw_costs))

In [95]:
cost_labels

tensor([0.9964], dtype=torch.float64)

In [96]:
QUERYID

15

In [97]:
# get the node features
df

,EXPLAIN_TIME,OPERATOR_ID,OPERATOR_TYPE,PREDICATE1,PREDICATE2,PREDICATE3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3,JOIN_KEY,SOURCE_ID,TABLE,PARSED_PRED1,PARSED_PRED2,PARSED_PRED3,HEIGHT
0,2024-12-03-20.44.07.528574,1,RETURN,NaN,NaN,NaN,1.000000,1.000000,1.0,NaN,NaN,NaN,None,None,None,11
1,2024-12-03-20.44.07.528574,2,TQ,NaN,NaN,NaN,1.000000,1.000000,1.0,NaN,NaN,NaN,None,None,None,10
2,2024-12-03-20.44.07.528574,3,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_INFO.MOVIE_ID = AHNAF.TITLE.ID,NaN,NaN,None,None,None,9
3,2024-12-03-20.44.07.528574,4,TBSCAN,AHNAF.TITLE.PRODUCTION_YEAR <= 2020,AHNAF.TITLE.PRODUCTION_YEAR >= 1950,None,0.971485,0.894801,1.0,NaN,-1.0,AHNAF.TITLE,"(AHNAF.TITLE.PRODUCTION_YEAR, <=, 2020)","(AHNAF.TITLE.PRODUCTION_YEAR, >=, 1950)",None,1
4,2024-12-03-20.44.07.528574,5,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.MOVIE_ID = AHNAF.MOVIE_INFO.MOVIE_ID,NaN,NaN,None,None,None,8
5,2024-12-03-20.44.07.528574,6,TBSCAN,"AHNAF.MOVIE_INFO.INFO IN ('Sweden ', 'Norway ', 'Germany ', 'Denmark ', 'Swedish ', 'Denish ', 'Norwegian ', 'German ', 'USA ', 'CANADA ', 'Netherlands ', 'Brazil ', 'UK ', 'Belgium ', 'Finland ', 'Hungary ', 'Estonia ', 'worldwide ', 'Australia ', 'Spain ', 'France ', 'Japan ', 'Columbia ', 'Slovenia ', 'Israel ', 'Venezuela ', 'Nigeria ', 'Philippines ', 'New Zealand ', 'Ireland ', 'Romania ', 'Non-USA ', 'Bulgaria ', 'Argentina ', 'Malaysia ', 'Singapore ', 'Turkey ', 'Sri Lanka ', 'Italy ', 'Indonesia ', 'South Korea ', 'Vietnam ', 'Slovakia ', 'Czech Republic ', 'China ', 'Portugal ', 'Greece ', 'Republic of Macedonia', 'Serbia ', 'Jamaica ', 'Switzerland ', 'Yugoslavia ', 'Mexico ', 'Austria ', 'Russia ')",None,None,0.029517,1.000000,1.0,NaN,-1.0,AHNAF.MOVIE_INFO,"(AHNAF.MOVIE_INFO.INFO, IN, [Sweden, Norway, Germany, Denmark, Swedish, Denish, Norwegian, German, USA, CANADA, Netherlands, Brazil, UK, Belgium, Finland, Hungary, Estonia, worldwide, Australia, Spain, France, Japan, Columbia, Slovenia, Israel, Venezuela, Nigeria, Philippines, New Zealand, Ireland, Romania, Non-USA, Bulgaria, Argentina, Malaysia, Singapore, Turkey, Sri Lanka, Italy, Indonesia, South Korea, Vietnam, Slovakia, Czech Republic, China, Portugal, Greece, Republic of Macedonia, Serbia, Jamaica, Switzerland, Yugoslavia, Mexico, Austria, Russia])",None,None,1
6,2024-12-03-20.44.07.528574,7,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.COMPANY_ID = AHNAF.COMPANY_NAME.ID,NaN,NaN,None,None,None,7
7,2024-12-03-20.44.07.528574,8,TBSCAN,AHNAF.COMPANY_NAME.COUNTRY_CODE = '[us]',None,None,0.359728,1.000000,1.0,NaN,-1.0,AHNAF.COMPANY_NAME,"(AHNAF.COMPANY_NAME.COUNTRY_CODE, =, '[us]')",None,None,1
8,2024-12-03-20.44.07.528574,9,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.COMPANY_TYPE_ID = AHNAF.COMPANY_TYPE.ID,NaN,NaN,None,None,None,6
9,2024-12-03-20.44.07.528574,10,HSJOIN,NaN,NaN,NaN,1.000000,1.000000,1.0,AHNAF.MOVIE_LINK.MOVIE_ID = AHNAF.MOVIE_COMPANIES.MOVIE_ID,NaN,NaN,None,None,None,5


In [98]:
df.shape

(19, 16)

In [99]:
df.columns

Index(['EXPLAIN_TIME', 'OPERATOR_ID', 'OPERATOR_TYPE', 'PREDICATE1',
       'PREDICATE2', 'PREDICATE3', 'FILTER_FACTOR1', 'FILTER_FACTOR2',
       'FILTER_FACTOR3', 'JOIN_KEY', 'SOURCE_ID', 'TABLE', 'PARSED_PRED1',
       'PARSED_PRED2', 'PARSED_PRED3', 'HEIGHT'],
      dtype='object')

In [100]:
df_node_feat = df[['OPERATOR_ID', 'OPERATOR_TYPE', 'TABLE', 'PREDICATE1', 'PREDICATE2', 'PREDICATE3', 'PARSED_PRED1', 'PARSED_PRED2', 'PARSED_PRED3', 'FILTER_FACTOR1', 'FILTER_FACTOR2', 'FILTER_FACTOR3', 'JOIN_KEY', 'HEIGHT']]

In [101]:
df_node_feat.sample(3)

,OPERATOR_ID,OPERATOR_TYPE,TABLE,PREDICATE1,PREDICATE2,PREDICATE3,PARSED_PRED1,PARSED_PRED2,PARSED_PRED3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3,JOIN_KEY,HEIGHT
5,6,TBSCAN,AHNAF.MOVIE_INFO,"AHNAF.MOVIE_INFO.INFO IN ('Sweden ', 'Norway ', 'Germany ', 'Denmark ', 'Swedish ', 'Denish ', 'Norwegian ', 'German ', 'USA ', 'CANADA ', 'Netherlands ', 'Brazil ', 'UK ', 'Belgium ', 'Finland ', 'Hungary ', 'Estonia ', 'worldwide ', 'Australia ', 'Spain ', 'France ', 'Japan ', 'Columbia ', 'Slovenia ', 'Israel ', 'Venezuela ', 'Nigeria ', 'Philippines ', 'New Zealand ', 'Ireland ', 'Romania ', 'Non-USA ', 'Bulgaria ', 'Argentina ', 'Malaysia ', 'Singapore ', 'Turkey ', 'Sri Lanka ', 'Italy ', 'Indonesia ', 'South Korea ', 'Vietnam ', 'Slovakia ', 'Czech Republic ', 'China ', 'Portugal ', 'Greece ', 'Republic of Macedonia', 'Serbia ', 'Jamaica ', 'Switzerland ', 'Yugoslavia ', 'Mexico ', 'Austria ', 'Russia ')",None,None,"(AHNAF.MOVIE_INFO.INFO, IN, [Sweden, Norway, Germany, Denmark, Swedish, Denish, Norwegian, German, USA, CANADA, Netherlands, Brazil, UK, Belgium, Finland, Hungary, Estonia, worldwide, Australia, Spain, France, Japan, Columbia, Slovenia, Israel, Venezuela, Nigeria, Philippines, New Zealand, Ireland, Romania, Non-USA, Bulgaria, Argentina, Malaysia, Singapore, Turkey, Sri Lanka, Italy, Indonesia, South Korea, Vietnam, Slovakia, Czech Republic, China, Portugal, Greece, Republic of Macedonia, Serbia, Jamaica, Switzerland, Yugoslavia, Mexico, Austria, Russia])",None,None,0.029517,1.0,1.0,NaN,1
18,17,TBSCAN,AHNAF.KEYWORD,AHNAF.KEYWORD.KEYWORD = 'helicopter',None,None,"(AHNAF.KEYWORD.KEYWORD, =, 'helicopter')",None,None,0.000007,1.0,1.0,NaN,1
7,8,TBSCAN,AHNAF.COMPANY_NAME,AHNAF.COMPANY_NAME.COUNTRY_CODE = '[us]',None,None,"(AHNAF.COMPANY_NAME.COUNTRY_CODE, =, '[us]')",None,None,0.359728,1.0,1.0,NaN,1


In [102]:
# Generate the bitmap indicating NaN (0) or non-NaN (1) in PREDICATE1, PREDICATE2, and PREDICATE3
bitmap = df_node_feat[['PREDICATE1', 'PREDICATE2', 'PREDICATE3']].notna().astype(int).values.tolist()

# Display the bitmap
print(bitmap)

df_node_feat['PREDICATE_MASK'] = bitmap

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [1, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0], [0, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [1, 0, 0]]


In [103]:
df_node_feat.sample(5)

,OPERATOR_ID,OPERATOR_TYPE,TABLE,PREDICATE1,PREDICATE2,PREDICATE3,PARSED_PRED1,PARSED_PRED2,PARSED_PRED3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3,JOIN_KEY,HEIGHT,PREDICATE_MASK
4,5,HSJOIN,NaN,NaN,NaN,NaN,None,None,None,1.000000,1.000000,1.0,AHNAF.MOVIE_COMPANIES.MOVIE_ID = AHNAF.MOVIE_INFO.MOVIE_ID,8,"[0, 0, 0]"
3,4,TBSCAN,AHNAF.TITLE,AHNAF.TITLE.PRODUCTION_YEAR <= 2020,AHNAF.TITLE.PRODUCTION_YEAR >= 1950,None,"(AHNAF.TITLE.PRODUCTION_YEAR, <=, 2020)","(AHNAF.TITLE.PRODUCTION_YEAR, >=, 1950)",None,0.971485,0.894801,1.0,NaN,1,"[1, 1, 0]"
10,19,TBSCAN,AHNAF.COMPANY_TYPE,AHNAF.COMPANY_TYPE.KIND = 'distributors',None,None,"(AHNAF.COMPANY_TYPE.KIND, =, 'distributors')",None,None,0.250000,1.000000,1.0,NaN,1,"[1, 0, 0]"
9,10,HSJOIN,NaN,NaN,NaN,NaN,None,None,None,1.000000,1.000000,1.0,AHNAF.MOVIE_LINK.MOVIE_ID = AHNAF.MOVIE_COMPANIES.MOVIE_ID,5,"[0, 0, 0]"
17,16,TBSCAN,AHNAF.MOVIE_KEYWORD,NaN,NaN,NaN,None,None,None,1.000000,1.000000,1.0,NaN,1,"[0, 0, 0]"


In [104]:
df_node_feat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   OPERATOR_ID     19 non-null     int64  
 1   OPERATOR_TYPE   19 non-null     object 
 2   TABLE           9 non-null      object 
 3   PREDICATE1      7 non-null      object 
 4   PREDICATE2      1 non-null      object 
 5   PREDICATE3      0 non-null      object 
 6   PARSED_PRED1    7 non-null      object 
 7   PARSED_PRED2    1 non-null      object 
 8   PARSED_PRED3    0 non-null      object 
 9   FILTER_FACTOR1  19 non-null     float64
 10  FILTER_FACTOR2  19 non-null     float64
 11  FILTER_FACTOR3  19 non-null     float64
 12  JOIN_KEY        8 non-null      object 
 13  HEIGHT          19 non-null     int64  
 14  PREDICATE_MASK  19 non-null     object 
dtypes: float64(3), int64(2), object(10)
memory usage: 2.4+ KB


In [105]:
df_node_feat.sample(3)

,OPERATOR_ID,OPERATOR_TYPE,TABLE,PREDICATE1,PREDICATE2,PREDICATE3,PARSED_PRED1,PARSED_PRED2,PARSED_PRED3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3,JOIN_KEY,HEIGHT,PREDICATE_MASK
4,5,HSJOIN,NaN,NaN,NaN,NaN,None,None,None,1.000000,1.0,1.0,AHNAF.MOVIE_COMPANIES.MOVIE_ID = AHNAF.MOVIE_INFO.MOVIE_ID,8,"[0, 0, 0]"
1,2,TQ,NaN,NaN,NaN,NaN,None,None,None,1.000000,1.0,1.0,NaN,10,"[0, 0, 0]"
18,17,TBSCAN,AHNAF.KEYWORD,AHNAF.KEYWORD.KEYWORD = 'helicopter',None,None,"(AHNAF.KEYWORD.KEYWORD, =, 'helicopter')",None,None,0.000007,1.0,1.0,NaN,1,"[1, 0, 0]"


In [106]:
# Function to collect predicate and parsed predicate pairs
def gather_predicate_info(row):
    gathered_data = []
    for i in range(1, 4):  # Loop through PREDICATE1, PREDICATE2, PREDICATE3
        predicate_col = f"PREDICATE{i}"
        parsed_col = f"PARSED_PRED{i}"
        if pd.notna(row[predicate_col]):  # Only include non-None values
            gathered_data.append({'PREDICATE': row[predicate_col], 'PARSED_PRED': row[parsed_col]})
    return gathered_data

# Apply the function to collect all predicates and parsed predicates
all_predicates = []
df_node_feat.apply(lambda row: all_predicates.extend(gather_predicate_info(row)), axis=1)

# Create a new DataFrame from the collected data
df_predicate_pairs = pd.DataFrame(all_predicates)

# Display the new DataFrame
#print(df_predicate_pairs)

In [107]:
df_predicate_pairs

,PREDICATE,PARSED_PRED
0,AHNAF.TITLE.PRODUCTION_YEAR <= 2020,"(AHNAF.TITLE.PRODUCTION_YEAR, <=, 2020)"
1,AHNAF.TITLE.PRODUCTION_YEAR >= 1950,"(AHNAF.TITLE.PRODUCTION_YEAR, >=, 1950)"
2,"AHNAF.MOVIE_INFO.INFO IN ('Sweden ', 'Norway ', 'Germany ', 'Denmark ', 'Swedish ', 'Denish ', 'Norwegian ', 'German ', 'USA ', 'CANADA ', 'Netherlands ', 'Brazil ', 'UK ', 'Belgium ', 'Finland ', 'Hungary ', 'Estonia ', 'worldwide ', 'Australia ', 'Spain ', 'France ', 'Japan ', 'Columbia ', 'Slovenia ', 'Israel ', 'Venezuela ', 'Nigeria ', 'Philippines ', 'New Zealand ', 'Ireland ', 'Romania ', 'Non-USA ', 'Bulgaria ', 'Argentina ', 'Malaysia ', 'Singapore ', 'Turkey ', 'Sri Lanka ', 'Italy ', 'Indonesia ', 'South Korea ', 'Vietnam ', 'Slovakia ', 'Czech Republic ', 'China ', 'Portugal ', 'Greece ', 'Republic of Macedonia', 'Serbia ', 'Jamaica ', 'Switzerland ', 'Yugoslavia ', 'Mexico ', 'Austria ', 'Russia ')","(AHNAF.MOVIE_INFO.INFO, IN, [Sweden, Norway, Germany, Denmark, Swedish, Denish, Norwegian, German, USA, CANADA, Netherlands, Brazil, UK, Belgium, Finland, Hungary, Estonia, worldwide, Australia, Spain, France, Japan, Columbia, Slovenia, Israel, Venezuela, Nigeria, Philippines, New Zealand, Ireland, Romania, Non-USA, Bulgaria, Argentina, Malaysia, Singapore, Turkey, Sri Lanka, Italy, Indonesia, South Korea, Vietnam, Slovakia, Czech Republic, China, Portugal, Greece, Republic of Macedonia, Serbia, Jamaica, Switzerland, Yugoslavia, Mexico, Austria, Russia])"
3,AHNAF.COMPANY_NAME.COUNTRY_CODE = '[us]',"(AHNAF.COMPANY_NAME.COUNTRY_CODE, =, '[us]')"
4,AHNAF.COMPANY_TYPE.KIND = 'distributors',"(AHNAF.COMPANY_TYPE.KIND, =, 'distributors')"
5,AHNAF.MOVIE_COMPANIES.NOTE IS NULL,"(AHNAF.MOVIE_COMPANIES.NOTE, IS, NULL)"
6,AHNAF.LINK_TYPE.LINK LIKE '%featured in%',"(AHNAF.LINK_TYPE.LINK, LIKE, '%featured in%')"
7,AHNAF.KEYWORD.KEYWORD = 'helicopter',"(AHNAF.KEYWORD.KEYWORD, =, 'helicopter')"


In [108]:
# Function to transform PARSED_PRED into a sentence
def construct_sentence(parsed_pred):
    if not isinstance(parsed_pred, tuple) or len(parsed_pred) != 3:
        return None  # Skip invalid entries

    col, op, val = parsed_pred
    # Convert list values in 'val' to a string representation
    if isinstance(val, list):
        val = ", ".join(map(str, val))

    # Construct the sentence
    return f"'col: {col} [SEP] op: {op} [SEP] val: {val}'"

# Apply the transformation to the DataFrame
df_predicate_pairs['SENTENCE'] = df_predicate_pairs['PARSED_PRED'].apply(construct_sentence)

In [109]:
df_predicate_pairs.sample(3)

,PREDICATE,PARSED_PRED,SENTENCE
1,AHNAF.TITLE.PRODUCTION_YEAR >= 1950,"(AHNAF.TITLE.PRODUCTION_YEAR, >=, 1950)",'col: AHNAF.TITLE.PRODUCTION_YEAR [SEP] op: >= [SEP] val: 1950'
7,AHNAF.KEYWORD.KEYWORD = 'helicopter',"(AHNAF.KEYWORD.KEYWORD, =, 'helicopter')",'col: AHNAF.KEYWORD.KEYWORD [SEP] op: = [SEP] val: 'helicopter''
0,AHNAF.TITLE.PRODUCTION_YEAR <= 2020,"(AHNAF.TITLE.PRODUCTION_YEAR, <=, 2020)",'col: AHNAF.TITLE.PRODUCTION_YEAR [SEP] op: <= [SEP] val: 2020'


In [110]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each sentence
df_predicate_pairs['EMBEDDING'] = df_predicate_pairs['SENTENCE'].apply(lambda sentence: model.encode(sentence))

2024-12-09 18:34:49.632282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733798089.669279  854004 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733798089.679413  854004 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 18:34:49.724916: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [111]:
df_predicate_pairs.head(1)

,PREDICATE,PARSED_PRED,SENTENCE,EMBEDDING
0,AHNAF.TITLE.PRODUCTION_YEAR <= 2020,"(AHNAF.TITLE.PRODUCTION_YEAR, <=, 2020)",'col: AHNAF.TITLE.PRODUCTION_YEAR [SEP] op: <= [SEP] val: 2020',"[-0.030363174, 0.048389953, -0.021899467, -0.005240833, -0.024279403, 0.020551767, -0.012246654, -0.039943308, -0.02386397, -0.015855676, 0.0069852807, -0.047658853, -0.01638193, -0.04722373, -0.11423087, 0.0032685199, -0.0795343, -0.022365965, 0.045112208, -0.057909798, -0.023976313, 0.064009055, -0.0069753737, 0.05262889, 0.02415314, 0.002587879, -0.06784337, 0.096567914, -0.026066557, -0.025283795, -0.016618578, 0.09429871, 0.055063874, -0.005456747, 0.072915815, -0.05642533, -0.08098891, -0.056052167, -0.019505966, 0.03844296, 0.0065593747, -0.12428242, 0.016875943, 0.0050946563, 0.005787534, -0.021006765, 0.019321391, -0.06971221, 0.050681908, 0.084798485, -0.08250859, -0.02268022, -0.034729276, -0.069544636, 0.0099973725, 0.06278284, -0.018374965, -0.012056542, 0.024473375, -0.010750123, -0.06742693, 0.018134657, 0.0052611274, -0.011520891, -0.011218971, -0.049683478, 0.014459895, -0.00014206476, -0.10637116, 0.017658133, 0.06005995, -0.05025352, -0.06280807, -0.02455058, 0.005408528, 0.059128042, 0.038508657, -0.03790531, 0.11181203, 0.0013798135, 0.011719071, -0.11990799, -0.065902345, -0.018969446, 0.0068259547, 0.0047359196, 0.028232211, -0.06326384, 0.03912953, -0.002533122, -0.08695435, -0.055635702, 0.061895724, 0.06425125, -0.08939428, 0.04556702, 0.02007965, 0.039445587, 0.022460211, 0.08817751, ...]"


In [112]:
# Create the dictionary
predicate_embedding_dict = dict(zip(df_predicate_pairs['PREDICATE'], df_predicate_pairs['EMBEDDING']))

# Display the dictionary
for predicate, embedding in predicate_embedding_dict.items():
    print(f"Predicate: {predicate}")
    print(f"Embedding: {embedding}")
    print()


Predicate: AHNAF.TITLE.PRODUCTION_YEAR <= 2020
Embedding: [-3.03631742e-02  4.83899526e-02 -2.18994673e-02 -5.24083292e-03
 -2.42794026e-02  2.05517672e-02 -1.22466544e-02 -3.99433076e-02
 -2.38639694e-02 -1.58556756e-02  6.98528066e-03 -4.76588532e-02
 -1.63819306e-02 -4.72237282e-02 -1.14230871e-01  3.26851988e-03
 -7.95342997e-02 -2.23659649e-02  4.51122075e-02 -5.79097979e-02
 -2.39763130e-02  6.40090555e-02 -6.97537372e-03  5.26288897e-02
  2.41531394e-02  2.58787908e-03 -6.78433701e-02  9.65679139e-02
 -2.60665566e-02 -2.52837949e-02 -1.66185778e-02  9.42987129e-02
  5.50638735e-02 -5.45674702e-03  7.29158148e-02 -5.64253293e-02
 -8.09889063e-02 -5.60521670e-02 -1.95059665e-02  3.84429581e-02
  6.55937474e-03 -1.24282420e-01  1.68759432e-02  5.09465626e-03
  5.78753417e-03 -2.10067648e-02  1.93213914e-02 -6.97122067e-02
  5.06819077e-02  8.47984850e-02 -8.25085863e-02 -2.26802193e-02
 -3.47292759e-02 -6.95446357e-02  9.99737252e-03  6.27828389e-02
 -1.83749646e-02 -1.20565416e-02

In [113]:
# Get the embedding dimension from the pretrained model
sample_embedding = model.encode(["test"])[0]  # Generate an example embedding
embedding_dim = sample_embedding.shape[0]     # Determine the dimension

# Define a zero embedding vector
zero_embedding = np.zeros(embedding_dim)

# Updated function to fetch embeddings
def get_embedding(predicate, embedding_dict):
    if predicate is not None and predicate in embedding_dict:
        return embedding_dict[predicate]
    return zero_embedding  # Return a zero vector if predicate is None or not in the dictionary

# Add EMBEDDING1, EMBEDDING2, and EMBEDDING3 columns to df_node_feat
df_node_feat['EMBEDDING1'] = df_node_feat['PREDICATE1'].apply(lambda pred: get_embedding(pred, predicate_embedding_dict))
df_node_feat['EMBEDDING2'] = df_node_feat['PREDICATE2'].apply(lambda pred: get_embedding(pred, predicate_embedding_dict))
df_node_feat['EMBEDDING3'] = df_node_feat['PREDICATE3'].apply(lambda pred: get_embedding(pred, predicate_embedding_dict))

# Display the updated DataFrame
print(df_node_feat[['PREDICATE1', 'EMBEDDING1', 'PREDICATE2', 'EMBEDDING2', 'PREDICATE3', 'EMBEDDING3']])

In [114]:
df_node_feat.sample(1)

,OPERATOR_ID,OPERATOR_TYPE,TABLE,PREDICATE1,PREDICATE2,PREDICATE3,PARSED_PRED1,PARSED_PRED2,PARSED_PRED3,FILTER_FACTOR1,FILTER_FACTOR2,FILTER_FACTOR3,JOIN_KEY,HEIGHT,PREDICATE_MASK,EMBEDDING1,EMBEDDING2,EMBEDDING3
3,4,TBSCAN,AHNAF.TITLE,AHNAF.TITLE.PRODUCTION_YEAR <= 2020,AHNAF.TITLE.PRODUCTION_YEAR >= 1950,None,"(AHNAF.TITLE.PRODUCTION_YEAR, <=, 2020)","(AHNAF.TITLE.PRODUCTION_YEAR, >=, 1950)",None,0.971485,0.894801,1.0,NaN,1,"[1, 1, 0]","[-0.030363174, 0.048389953, -0.021899467, -0.005240833, -0.024279403, 0.020551767, -0.012246654, -0.039943308, -0.02386397, -0.015855676, 0.0069852807, -0.047658853, -0.01638193, -0.04722373, -0.11423087, 0.0032685199, -0.0795343, -0.022365965, 0.045112208, -0.057909798, -0.023976313, 0.064009055, -0.0069753737, 0.05262889, 0.02415314, 0.002587879, -0.06784337, 0.096567914, -0.026066557, -0.025283795, -0.016618578, 0.09429871, 0.055063874, -0.005456747, 0.072915815, -0.05642533, -0.08098891, -0.056052167, -0.019505966, 0.03844296, 0.0065593747, -0.12428242, 0.016875943, 0.0050946563, 0.005787534, -0.021006765, 0.019321391, -0.06971221, 0.050681908, 0.084798485, -0.08250859, -0.02268022, -0.034729276, -0.069544636, 0.0099973725, 0.06278284, -0.018374965, -0.012056542, 0.024473375, -0.010750123, -0.06742693, 0.018134657, 0.0052611274, -0.011520891, -0.011218971, -0.049683478, 0.014459895, -0.00014206476, -0.10637116, 0.017658133, 0.06005995, -0.05025352, -0.06280807, -0.02455058, 0.005408528, 0.059128042, 0.038508657, -0.03790531, 0.11181203, 0.0013798135, 0.011719071, -0.11990799, -0.065902345, -0.018969446, 0.0068259547, 0.0047359196, 0.028232211, -0.06326384, 0.03912953, -0.002533122, -0.08695435, -0.055635702, 0.061895724, 0.06425125, -0.08939428, 0.04556702, 0.02007965, 0.039445587, 0.022460211, 0.08817751, ...]","[-0.04063183, 0.04969196, -0.0659921, -0.035999417, -0.01856833, 0.027513998, -0.015516212, -0.035694327, -0.03599221, -0.039366566, 0.014671134, -0.020138657, 0.019307451, -0.03986847, -0.11315486, -0.021278467, -0.071898915, 0.012243174, 0.046157174, -0.047082305, -0.059845433, 0.054360505, 0.01736014, 0.040442348, 0.03696225, 0.016762292, -0.05641205, 0.10353774, -0.04218849, -0.037193593, -0.01784175, 0.112318724, 0.077110246, -0.030671796, 0.07007865, -0.05939632, -0.08393267, -0.05174786, -0.0563708, 0.033389427, -0.031071082, -0.11430907, -0.022772793, 0.008685776, 0.029715069, -0.006692144, 0.013999563, -0.063133754, 0.038642224, 0.08767199, -0.07094794, 0.018743586, -0.040975023, -0.0634717, 0.0051295203, 0.050050873, 0.004135242, 0.0058441125, 0.004157504, -0.00039766257, -0.11051997, 0.0038261374, -0.0067373286, -0.0118650645, -0.0184575, -0.03651956, 0.010031634, -0.033792492, -0.102381416, -0.02100128, 0.005323957, -0.04260244, -0.053805374, -0.03247822, 0.004550078, 0.028622227, 0.019549128, -0.06390744, 0.069217585, -0.034907266, 0.035772245, -0.12680408, -0.09140451, -0.01931074, 0.001914587, 0.014863714, 0.0603899, -0.051766418, 0.035411574, -0.00719733, -0.10143026, -0.026564777, 0.0609764, 0.058224864, -0.046479847, 0.04314641, 0.015378297, 0.0735212, 0.046451755, 0.074447125, ...]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"


In [115]:
df_node_feat.columns

Index(['OPERATOR_ID', 'OPERATOR_TYPE', 'TABLE', 'PREDICATE1', 'PREDICATE2',
       'PREDICATE3', 'PARSED_PRED1', 'PARSED_PRED2', 'PARSED_PRED3',
       'FILTER_FACTOR1', 'FILTER_FACTOR2', 'FILTER_FACTOR3', 'JOIN_KEY',
       'HEIGHT', 'PREDICATE_MASK', 'EMBEDDING1', 'EMBEDDING2', 'EMBEDDING3'],
      dtype='object')

In [116]:
# Helper function for label encoding with NaN values set to 0
def label_encode_with_nan(value, encoder, counter_start=1):
    if pd.isna(value):
        return 0, counter_start  # Return 0 for NaN and do not increment the counter
    if value not in encoder:
        encoder[value] = counter_start
        counter_start += 1
    return encoder[value], counter_start

### Load or Initialize Encoding Dictionaries

In [117]:
import pickle
import os

# Paths for dictionary storage
dictionary_dir = "./dictionaries"
os.makedirs(dictionary_dir, exist_ok=True)
op_type_file = os.path.join(dictionary_dir, "op_type_encoder.pkl")
table_file = os.path.join(dictionary_dir, "table_encoder.pkl")
join_key_file = os.path.join(dictionary_dir, "join_key_encoder.pkl")

# Load or initialize dictionaries
def load_or_initialize(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    return {}

op_type_encoder = load_or_initialize(op_type_file)
table_encoder = load_or_initialize(table_file)
join_key_encoder = load_or_initialize(join_key_file)

### Perform Encoding

In [118]:
# Counters for label encoding
op_type_counter, table_counter, join_key_counter = len(op_type_encoder) + 1, len(table_encoder) + 1, len(join_key_encoder) + 1
encoded_operator_type = []
encoded_table = []
encoded_join_key = []

for op, tbl, jk in zip(df_node_feat['OPERATOR_TYPE'], df_node_feat['TABLE'], df_node_feat['JOIN_KEY']):
    encoded_op, op_type_counter = label_encode_with_nan(op, op_type_encoder, op_type_counter)
    encoded_operator_type.append(encoded_op)
    
    encoded_tbl, table_counter = label_encode_with_nan(tbl, table_encoder, table_counter)
    encoded_table.append(encoded_tbl)
    
    encoded_jk, join_key_counter = label_encode_with_nan(jk, join_key_encoder, join_key_counter)
    encoded_join_key.append(encoded_jk)

df_node_feat['ENCODED_OPERATOR_TYPE'] = encoded_operator_type
df_node_feat['ENCODED_TABLE'] = encoded_table
df_node_feat['ENCODED_JOIN_KEY'] = encoded_join_key

### Save Dictionaries


In [119]:
# Save updated dictionaries back to files
with open(op_type_file, 'wb') as f:
    pickle.dump(op_type_encoder, f)
with open(table_file, 'wb') as f:
    pickle.dump(table_encoder, f)
with open(join_key_file, 'wb') as f:
    pickle.dump(join_key_encoder, f)

### Display Encoding Results and Dictionaries

In [120]:
# Inspect encoded columns and dictionaries
print(df_node_feat[['OPERATOR_TYPE', 'TABLE', 'JOIN_KEY', 'ENCODED_OPERATOR_TYPE', 'ENCODED_TABLE', 'ENCODED_JOIN_KEY']])
print("\nOPERATOR_TYPE Encoding Dictionary:", op_type_encoder)
print("TABLE Encoding Dictionary:", table_encoder)
print("JOIN_KEY Encoding Dictionary:", join_key_encoder)

   OPERATOR_TYPE                  TABLE  \
0         RETURN                    NaN   
1         TQ                        NaN   
2         HSJOIN                    NaN   
3         TBSCAN            AHNAF.TITLE   
4         HSJOIN                    NaN   
5         TBSCAN       AHNAF.MOVIE_INFO   
6         HSJOIN                    NaN   
7         TBSCAN     AHNAF.COMPANY_NAME   
8         HSJOIN                    NaN   
9         HSJOIN                    NaN   
10        TBSCAN     AHNAF.COMPANY_TYPE   
11        TBSCAN  AHNAF.MOVIE_COMPANIES   
12        HSJOIN                    NaN   
13        HSJOIN                    NaN   
14        TBSCAN        AHNAF.LINK_TYPE   
15        TBSCAN       AHNAF.MOVIE_LINK   
16        HSJOIN                    NaN   
17        TBSCAN    AHNAF.MOVIE_KEYWORD   
18        TBSCAN          AHNAF.KEYWORD   

                                                         JOIN_KEY  \
0                                                             NaN   
1

In [121]:
df_node_feat[['ENCODED_OPERATOR_TYPE', 'ENCODED_TABLE', 'ENCODED_JOIN_KEY', 'PREDICATE_MASK']]

,ENCODED_OPERATOR_TYPE,ENCODED_TABLE,ENCODED_JOIN_KEY,PREDICATE_MASK
0,1,0,0,"[0, 0, 0]"
1,2,0,0,"[0, 0, 0]"
2,3,0,1,"[0, 0, 0]"
3,4,1,0,"[1, 1, 0]"
4,3,0,2,"[0, 0, 0]"
5,4,2,0,"[1, 0, 0]"
6,3,0,3,"[0, 0, 0]"
7,4,3,0,"[1, 0, 0]"
8,3,0,4,"[0, 0, 0]"
9,3,0,5,"[0, 0, 0]"


# Making a tensor of node features

In [122]:
df_node_feat.columns

Index(['OPERATOR_ID', 'OPERATOR_TYPE', 'TABLE', 'PREDICATE1', 'PREDICATE2',
       'PREDICATE3', 'PARSED_PRED1', 'PARSED_PRED2', 'PARSED_PRED3',
       'FILTER_FACTOR1', 'FILTER_FACTOR2', 'FILTER_FACTOR3', 'JOIN_KEY',
       'HEIGHT', 'PREDICATE_MASK', 'EMBEDDING1', 'EMBEDDING2', 'EMBEDDING3',
       'ENCODED_OPERATOR_TYPE', 'ENCODED_TABLE', 'ENCODED_JOIN_KEY'],
      dtype='object')

In [123]:
len(df_node_feat)

19

In [124]:
tensor_data = []
for _, row in df_node_feat.iterrows():
    # Flatten all values into a single list
    flat_row = np.concatenate([
        [row['ENCODED_OPERATOR_TYPE']],
        [row['ENCODED_JOIN_KEY']],
        [row['ENCODED_TABLE']],
        row['PREDICATE_MASK'],
        row['EMBEDDING1'],  # Directly use the embedding column
        row['EMBEDDING2'],  # Directly use the embedding column
        row['EMBEDDING3'],  # Directly use the embedding column
        [row['FILTER_FACTOR1']],
        [row['FILTER_FACTOR2']],
        [row['FILTER_FACTOR3']]
    ])
    tensor_data.append(flat_row)

# Convert the list of concatenated rows into a tensor
x = torch.tensor(tensor_data, dtype=torch.float32)
x = x.unsqueeze(0)


In [125]:
x.shape

torch.Size([1, 19, 1161])

In [126]:
def pad_2d_unsqueeze(x, padlen):
    # dont know why add 1, comment out first
#    x = x + 1 # pad id = 0
    _, xlen, xdim = x.size()
    if xlen < padlen:
        new_x = x.new_zeros([padlen, xdim], dtype=x.dtype) + 1
        new_x[:xlen, :] = x
        x = new_x
    return x.unsqueeze(0)

def pad_1d_unsqueeze(x, padlen):
    x = x + 1 # pad id = 0
    xlen = x.size(0)
    if xlen < padlen:
        new_x = x.new_zeros([padlen], dtype=x.dtype)
        new_x[:xlen] = x
        x = new_x
    return x.unsqueeze(0)

In [127]:
x = pad_2d_unsqueeze(x, max_node)

In [128]:
x.shape

torch.Size([1, 30, 1161])

In [129]:
heights.shape

torch.Size([1, 30])

In [130]:
attn_bias.shape

torch.Size([1, 31, 31])

In [131]:
attn_bias

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, -inf, 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
       

## Define Model

In [132]:
class Args:
    # bs = 1024
    # SQ: smaller batch size
    bs = 1
    #lr = 0.001
    lr = 0.001
    # epochs = 200
    epochs = 50
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    # device = 'cuda:0'
    device = 'cpu'
    newpath = 'job_queries_training'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [133]:
from model.model import QueryFormer

model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = False, use_hist = False, \
                 pred_hid = args.pred_hid
                )

In [134]:
from model.dataset import PlanTreeDataset

In [135]:
cost_labels

tensor([0.9964], dtype=torch.float64)

In [136]:
raw_costs

[66821]

In [137]:
type(x)

torch.Tensor

In [138]:
type(rel_pos)

torch.Tensor

In [139]:
type(attn_bias)

torch.Tensor

In [140]:
type(heights)

torch.Tensor

In [141]:
type(cost_labels)

torch.Tensor

In [142]:
type(raw_costs)

list

In [143]:
x.shape

torch.Size([1, 30, 1161])

In [144]:
attn_bias.shape

torch.Size([1, 31, 31])

In [145]:
type(x)

torch.Tensor

In [146]:
rel_pos.shape

torch.Size([1, 30, 30])

In [147]:
heights.shape

torch.Size([1, 30])

In [148]:
cost_labels.shape

torch.Size([1])

In [149]:
type(raw_costs)

list

In [150]:
raw_costs

[66821]

## TEST - making multiple copies of the training sample

In [151]:
num_copies = 2

# Replicate data
replicated_x = [x.clone() for _ in range(num_copies)]
replicated_attn_bias = [attn_bias.clone() for _ in range(num_copies)]
replicated_rel_pos = [rel_pos.clone() for _ in range(num_copies)]
replicated_heights = [heights.clone() for _ in range(num_copies)]
replicated_cost_labels = [cost_labels.clone() for _ in range(num_copies)]  # Tensor replicated
replicated_raw_costs = raw_costs * num_copies  # List replicated

# Initialize dataset
dataset = PlanTreeDataset(
    num_copies,
    replicated_x,
    replicated_attn_bias,
    replicated_rel_pos,
    replicated_heights,
    replicated_cost_labels,
    replicated_raw_costs
)

print("Dataset length:", len(dataset))
print("First sample:", dataset[0])

Dataset length: 2
First sample: ({'x': tensor([[[1., 0., 0.,  ..., 1., 1., 1.],
         [2., 0., 0.,  ..., 1., 1., 1.],
         [3., 1., 0.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]]), 'attn_bias': tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0

In [152]:
assert len(replicated_x) == num_copies
assert len(replicated_attn_bias) == num_copies
assert len(replicated_rel_pos) == num_copies
assert len(replicated_heights) == num_copies
assert len(replicated_cost_labels) == num_copies
assert len(replicated_raw_costs) == num_copies

In [153]:
# Initialize the PlanTreeDataset with optional costs
#dataset = PlanTreeDataset(1, [x], [attn_bias], [rel_pos], [heights], cost_labels, raw_costs)


In [154]:
# Get the length of the dataset
print(f"Dataset size: {len(dataset)}")

# Access a single sample
sample, label = dataset[0]

# Print the sample contents
print("Sample contents:")
print("Feature Matrix (x):", sample['x'].shape)
print("Attention Bias (attn_bias):", sample['attn_bias'].shape)
print("Relative Positions (rel_pos):", sample['rel_pos'].shape)
print("Heights (heights):", sample['heights'].shape)
print("Label:", label)

Dataset size: 2
Sample contents:
Feature Matrix (x): torch.Size([1, 30, 1161])
Attention Bias (attn_bias): torch.Size([1, 31, 31])
Relative Positions (rel_pos): torch.Size([1, 30, 30])
Heights (heights): torch.Size([1, 30])
Label: (tensor([0.9964], dtype=torch.float64), tensor(66821))


In [155]:
# Example numpy label
import numpy as np
import torch.nn as nn
import importlib

from model import trainer
importlib.reload(trainer)
from  model.trainer import train_single, train


crit = nn.MSELoss()

# Train the model with the numpy label
# trained_model = train_single(model, dataset, dataset, crit, cost_norm, args)
model, best_model_path, train_embeddings, val_embeddings = train(model, dataset, dataset, crit, cost_norm, args)


Epoch: 0  Avg Loss: 0.09472102707479735, Time: 1.313530445098877
Median: 2.2384224267128037
Mean: 2.2384224267128037
Epoch: 20  Avg Loss: 1.2026972200374075e-06, Time: 31.078593730926514
Median: 1.0025860414110115
Mean: 1.0025860414110115
Epoch: 40  Avg Loss: 1.1098984664670297e-07, Time: 49.50162386894226
Median: 1.0008191531460695
Mean: 1.0008191531460695


In [156]:
print(f"Number of training embeddings (best epoch): {len(train_embeddings)}")
print(f"First training embedding shape: {train_embeddings[0].shape}")
print(f"Number of validation embeddings (best epoch): {len(val_embeddings)}")
print(f"First validation embedding shape: {val_embeddings[0].shape}")


Number of training embeddings (best epoch): 2
First training embedding shape: (1417,)
Number of validation embeddings (best epoch): 2
First validation embedding shape: (1417,)


In [162]:
print(f"First validation embedding shape: {val_embeddings[0]}")

First validation embedding shape: [-7.690985  -6.4473424  3.9361618 ... 11.051013   5.1709585 -8.524437 ]
